# Password Spray Investigation


<table border="1 solid black" align="left">
      <tbody>
        <tr>
          <td>Author</td>
          <td>Ian Hellen (@ianhellen, Twitter)</td>
        </tr>
        <tr>
          <td>Date</td>
          <td>November 12 2020</td>
        </tr>
        <tr>
          <td>Version</td>
          <td>1.0</td>
        </tr>
        <tr>
          <td>Requirements</td>
          <td>msticpy, hvplot</td>
        </tr>
      </tbody>
</table>


<hr>

## Overview

**Picking time periods and events to examine**
- Use Time Series analysis to detect unusual numbers of signin failures
  - Show anomalies by Signin Result Type
  - Analyze anomaly density and select subset of Result Types with strong clustering of failure events
  - Calculate time periods for these types
  - Allow user to select subset of available periods
  - Get data for failures and success logons in these selected periods
  
**Looking at features of failed signins**
1. Analyze Failed signins for repetitive patterns for UserAgent, AppID, DeviceDetails and other Properties
  - Pick features/properties based on repetitive nature vs. success signins
  - Try to find any successful logons that show the same values
2. Client IP Origin
  - Look at netblocks for client IPs - do failed logons come from same IP blocks as sucessful logins?
  - Highlight all failures coming from netblocks unused or rarely used in success logins.
3. Logon Request Timing
  - Do the failed logon events occur at specific min/second past hour - indicating an automated process initiating them.
  - Identify failures that have a regular timed pattern
4. IP Location data
  - Similar to netblocks - establish common locations for successful logons
  - Highlight all failed requests coming from unused or rare location
5. User Agent
  - Identify User Agents present in success logins
  - Identify all failed requests using UA that is not in this list
6. User Agent - AppId mapping
  - Look at frequency of pairing between User Agents and AppIds in failed and successful signins
7. Legacy authentication protocols
  - How often are legacy authentication protocols used in success vs. failed logins?
  - Are specific patterns in failed logons associated with a specific protocol
8. Pattern analysis in UA strings
  - This is not yet done.
9. IP Addresses with multiple names
  - It would be ususual for logons coming from a single IP address to be using different user names
  - Any that do this are flagged as suspicious
10. User names with multiple IP Addresses, UserAgents, Locations
  - While it is not unusual for users to be associated with multiple of these, seeing many of these may be suspicious
  - Failed logons with multiple values may indicated that the user name is 
    public knowlege and being attacked from multiple sources
11. IP Addresses identified as malicious by Azure Active Directory.
  - This information is usually only available to customers on a certain paid tier of AAD so we try not to depend on this.
  - If the information is available though, let's use it.

**Threat intelligence lookups for suspect IP addresses**
- We've collected suspicious IP addresses in many of the previous steps. We do threat intel
  lookups on those IPs to check for known bad origins

**Viewing the investigation results**
- Use above properties to cluster failed events based on features.
  - You can browse the sets clustered events - allowing large numbers of failure events
    to be consumed in manageable chunks<br>
- Observation browsing
  - The results from the feature analysis are collected together in a browseable list
    so that you don't have to trawl through the entired notebook to see the interesting data.<br>
- Results query
  - This is a simple interactive query interface allowing you to query subsets of rows
    and columns of the output data. You can also export the results of the query
    to the clipboard for analysis in other applications.

## Using the Notebook
The notebook contains a lot of details and illustrations of the analysis but most of
that is peripheral to actually checking your data for possible password spray events.
This additional analysis and visualization is left in the notebook for better explanation
of the techniques used - and make them easier to change/correct, if needed.

### Prerequisites
- msticpy - install the latest using `pip install --upgrade msticpy`
- hvplot - this will be installed by the `init_notebook` function in the following cell.

### Running the notebook
The quick and easy way of using the notebook is as follows:
- Individually run all of the cells up to the start of Part 1:
  - library initialization and installation
  - authenticating to the workspace
  - setting notebook parameters
  - setting the time boundaries for the analysis
- From the beginning of Part 1, select `Run selected cell and all below` from the notebook `Run` menu.
- Wait a while...
- Navigate to the summary browsers at the end of the notebook:
  - [Browse clustered events](#4.2-Browse-clustered-events)
  - [Browse investigation observations](#4.3-Browse-investigation-observations)
  - [Results interactive query](#4.4-Results-query-browser)

In [ ]:
import os
import sys
import warnings
from pathlib import Path

from IPython.display import HTML, Markdown, display

REQ_PYTHON_VER=(3, 6)
REQ_MSTICPY_VER=(0, 6, 0)

display(HTML("<h3>Starting Notebook setup...</h3>"))
if Path("./utils/nb_check.py").is_file():
    from utils.nb_check import check_mp_ver, check_python_ver

    check_python_ver(min_py_ver=REQ_PYTHON_VER)
    try:
        check_mp_ver(min_msticpy_ver=REQ_MSTICPY_VER)
    except ImportError:
        !pip install --upgrade msticpy
        if "msticpy" in sys.modules:
            importlib.reload(sys.modules["msticpy"])
        else:
            import msticpy
        check_mp_ver(REQ_MSTICPY_VER)

# If you are not launching this notebook from Azure Sentinel, install msticpy with
# !pip install msticpy

from msticpy.nbtools import nbinit

nbinit.init_notebook(
    namespace=globals(),
    extra_imports=[
        "ipwhois, IPWhois",
        "statistics, variance"
    ],
    additional_packages = ["hvplot"],
    friendly_exceptions=False,
)

import hvplot.pandas
from bokeh.io import output_notebook
from bokeh.plotting import reset_output, show

# pd.set_option("plotting.backend", "pandas_bokeh")
reset_output()
output_notebook()

In [ ]:
# Load Data provider and Authenticate
ws_config = WorkspaceConfig(workspace=YOUR_WORKSPACE_NAME)
qry_prov = QueryProvider(data_environment="LogAnalytics")
qry_prov.connect(connection_str=ws_config.code_connect_str)
table_index = qry_prov.schema_tables

## Notebook Settings

In [ ]:
from pathlib import Path


def use_caching() -> bool:
    """True if using caching for reference data."""
    return use_cache_cb.value


def mask_data() -> bool:
    """True if using data masking"""
    return mask_data_cb.value


def cache_folder() -> str:
    return cache_folder_txt.value


cwd = str(Path(".").resolve())
use_cache_cb = widgets.Checkbox(
    description="Use cached data", value=True, **WIDGET_DEFAULTS
)
cache_folder_txt = widgets.Text(
    description="Cache data folder", value=cwd, **WIDGET_DEFAULTS
)
mask_data_cb = widgets.Checkbox(
    description="Mask PII data", value=False, **WIDGET_DEFAULTS
)

md("Notebook setup options<hr>", "large, bold")
display(
    widgets.VBox(
        [use_cache_cb, cache_folder_txt, mask_data_cb], style={"border-style": "solid"}
    )
)
md("<hr>")

## Set time boundaries for analysis

In [ ]:
query_time = nbwidgets.QueryTime(
    units="day", max_before=60, max_after=30, before=30, after=0, auto_display=True
)

---
# Part 1 - Detecting anomalous logon events

---
## 1.1 Using Time Series to detect anomalous patterns in login failures

### Query Signing Logs for TimeSeries Anomalies for failure logons

In [ ]:
signin_ts_df = qry_prov.MultiDataSource.get_timeseries_anomalies(
    # "print",
    table="SigninLogs",
    start=query_time.start,
    end=query_time.end,
    timestampcolumn="TimeGenerated",
    aggregatecolumn="AppDisplayName",
    groupbycolumn="ResultType",
    aggregatefunction="count(AppDisplayName)",
    where_clause="| where ResultType != 0",
    add_query_items="| project-rename Total=AppDisplayName",
)

### Reference: AAD Signin Log Type Code descriptions.

In [ ]:
result_types = {
    "0": "Success",
    "50053": "Account is locked because user tried to sign in too many times with an incorrect user ID or password.",
    "50055": "Invalid password, entered expired password.",
    "50057": "User account is disabled. The account has been disabled by an administrator.",
    "50072": "Users' needs to enroll for second factor authentication (interactive).",
    "50074": "User did not pass the MFA challenge.",
    "50076": "User did not pass the MFA challenge (non interactive).",
    "50079": "User needs to enroll for second factor authentication.",
    "50097": "Device Authentication Required - DeviceId -DeviceAltSecId claims are null OR no device corresponding to the device identifier exists.",
    "50105": "The signed in user is not assigned to a role for the signed in application. Assign the user to the application. For more information: https://docs.microsoft.com/en-us/azure/active-directory/application-sign-in-problem-federated-sso-gallery#user-not-assigned-a-role.",
    "50126": "Invalid username or password or Invalid on-premise username or password.",
    "50155": "Device authentication failed for this user.",
    "53000": "Conditional Access policy requires a compliant device, and the device is not compliant. Have the user enroll their device with an approved MDM provider like Intune.",
    "53003": "Access has been blocked due to conditional access policies.",
}

result_type_categories = {
    "50053": "invalid_credentials",
    "50055": "invalid_credentials",
    "50057": "access_policy",
    "50072": "mfa",
    "50074": "mfa",
    "50076": "mfa",
    "50079": "mfa",
    "50097": "mfa",
    "50105": "access_policy",
    "50126": "invalid_credentials",
    "50155": "mfa",
    "53000": "access_policy",
    "53003": "access_policy",
}

client_apps = {
    "Browser": "modern",
    "Mobile Apps and Desktop clients": "modern",
    "Authenticated SMTP": "legacy",
    "AutoDiscover": "legacy",
    "Exchange ActiveSync": "legacy",
    "Exchange Online PowerShell": "legacy",
    "Exchange Web Services": "legacy",
    "IMAP4": "legacy",
    "MAPI Over HTTP": "legacy",
    "Offline Address Book": "legacy",
    "Other clients": "legacy",
    "Outlook Anywhere (RPC over HTTP)": "legacy",
    "POP3": "legacy",
    "Reporting Web Services": "legacy",
    "Universal Outlook": "legacy",
}


def legacy_clients() -> list:
    """Return a list of legacy client protocols."""
    return [app for app, cat in client_apps.items() if cat == "legacy"]


def is_legacy_client(client_app: str) -> bool:
    """Return True if client_app is legacy."""
    return client_apps.get(client_app) == "legacy"

## Showing Anomaly Graphs for all Result Types

Here we are showing time series anomaly charts for each Result Type.

Following this we will choose particular subsets of event types and time ranges to focus on.

In [ ]:
show_anom_plots = nbwidgets.OptionButtons(
    description="Show anomaly plots for all types?",
    buttons=["Yes", "No"],
    default="No",
    timeout=5,
)
await show_anom_plots.display_async()

In [ ]:
from msticpy.nbtools.timeseries import display_timeseries_anomolies

if show_anom_plots.value == "yes":
    plot_res_types = [res_id for res_id in result_types if res_id != "0"]

    for event_type in plot_res_types:
        ts_data = signin_ts_df[signin_ts_df["ResultType"] == event_type]
        if not ts_data.empty:
            plot = display_timeseries_anomolies(
                data=ts_data,
                title=f"Timeline anomalies for event type {event_type}: {result_types[event_type]}",
            )
        else:
            md(f"No events matching selected type {event_type}")

## 1.2 Narrow the data down to specific events and time ranges

If we combine all of the anomaly data on to a single timeline, you can 
see that there are periods with **higher density of events**. 
However, it is difficult to pick out these periods accurately.

Also, some Event/Result types have relatively few anomalies and may
not be interesting to look at further.

We're going to do some initial analysis to pick out potentally more
interesting ResultTypes and time ranges to look at.

In [ ]:
signin_ts_df[
    (signin_ts_df["ResultType"].isin([str(e_type) for e_type in plot_res_types]))
    & (signin_ts_df["anomalies"] == 1)
].mp_timeline.plot(source_columns=["ResultType"], group_by="ResultType", height=300);

### Calculate Anomaly density: which event types had high clustering with respect to time?

We want to get a number that gives us an indication of which event types have concentrated
bursts of anomalies, as opposed to a lot of anomalies that are distributed across the time range.

The intuition here is that password spray attacks are likely to be concentrated over a relatively
short period.

We calculate approximate **anomaly density** for each *Result Type* by:
- taking the time differences (in seconds) between individual anomaly instances for a give *ResultType*.
- then take the median value of these time differences differences.

The **density** for a given *ResultType* is calculated by dividing the total number anomalies by the median of time differences.

For N anomalies - a series with lots of shorter time intervals will have a higher
density score than a series that is more evenly distributed over time.

In [ ]:
event_type_scoring = []
for event_type in plot_res_types:
    event_type_score = {}
    ts_data = signin_ts_df[signin_ts_df["ResultType"] == event_type]
    anomalies = ts_data[ts_data.anomalies == 1]
    total_anomalies = len(anomalies)
    total_events = anomalies.Total.sum()
    if total_anomalies > 1:
        # Calculate median time difference between anomalies
        median_diff = anomalies.TimeGenerated.diff().median()
        median_sec_diff = median_diff.total_seconds()
    else:
        # If there is only one anomaly, set time difference of 24 hours
        median_sec_diff = 24 * 3600
    score = np.sum(np.abs(anomalies.score))

    event_type_score["event_type"] = event_type
    event_type_score["anom_count"] = total_events
    event_type_score["anom perc"] = total_events / len(ts_data)
    event_type_score["anom_total_score"] = score
    event_type_score["median_sec_diff"] = median_sec_diff
    event_type_score["anomaly_density"] = total_events / median_sec_diff
    event_type_scoring.append(event_type_score)

event_score_df = pd.DataFrame(event_type_scoring).sort_values(
    "anomaly_density", ascending=False
)
display(
    event_score_df.hvplot.bar(
        x="event_type",
        y="anomaly_density",
        title="Relative anomaly density by ResultType",
    )
)
display(event_score_df)

## Select Event Types to be used

We use the density calculation to select a set of events with the highest density.

#### Default selection is any *ResultType* with a density > 5% of the max *ResultType* density
We use this threshold to auto-select *ResultType*s in the subsequent cell.

In [ ]:
anom_density = widgets.FloatSlider(
    min=event_score_df.anomaly_density.min(),
    max=event_score_df.anomaly_density.max(),
    value=event_score_df.anomaly_density.max() * 0.05,
    step=0.0001,
    description="Anomaly density lower limit",
    layout=widgets.Layout(width="60%"),
    style={"description_width": "200px"},
    readout_format=".7f",
)
display(anom_density)

In [ ]:
def selected_event_types(fmt="list"):
    """
    Return list of currently selected events.

    Parameters
    ----------
    fmt: str
        Possible values are:
        - list == list of string values
        - int_list == list of ints
        - str == comma-separated values as string
        - q_str == like 'str' but values are quoted

    Returns
    -------
    list or str
        depending on `fmt` parameter
    """
    sel_events = wgt_event_type.value
    if fmt == "list":
        return [str(evt_id) for evt_id in sel_events]
    if fmt == "str":
        return ", ".join([str(evt_id) for evt_id in sel_events])
    if fmt == "int_list":
        return list(sel_events)
    if fmt == "q_str":
        return ", ".join([f"'{evt_id}'" for evt_id in sel_events])


def selected_events_label():
    """Return a string of selected event types for labelling saved data."""
    "_".join(selected_event_types())


# Create widgets
layout = widgets.Layout(width="50%", height="200px")
style = {"description_width": "200px"}

wgt_event_type = widgets.SelectMultiple(
    description="Select Result types to use",
    options=[
        (f"{evt_id}: ({result_type_categories.get(evt_id)}) {evt_desc}", evt_id)
        for evt_id, evt_desc in result_types.items()
        if evt_id != "0"
    ],
    layout=layout,
    style=style,
)

calc_types = list(
    event_score_df[event_score_df.anomaly_density > anom_density.value].event_type
)

categories = ["all", "calculated - based on anomaly density"] + sorted(
    list(set(result_type_categories.values()))
)

wgt_event_cat = widgets.RadioButtons(
    description="Select Result Types of category",
    options=categories,
    value="invalid_credentials",
    layout=layout,
    style=style,
)


def _set_by_category(change):
    sel_cat = change.get("new")
    if sel_cat == "all":
        wgt_event_type.value = [opt[1] for opt in wgt_event_type.options]
    elif sel_cat == "calculated":
        wgt_event_type.value = calc_types
    else:
        wgt_event_type.value = [
            evt_id for evt_id, cat in result_type_categories.items() if cat == sel_cat
        ]


_set_by_category({"new": "invalid_credentials"})
wgt_event_cat.observe(_set_by_category, names="value")


md(
    """
<p style='background-color: beige; font-size: medium'>
<b>Tip</b> - Performing analysis with multiple result categories can create confusing results.<br>
We recommend you select one or two Result Types at a time and
run through the notebook with these subsets.<br>
Alternatively, use the category selector and pick a category. Start with "invalid_credentials".

</p>
"""
)

display(widgets.HBox([wgt_event_type, wgt_event_cat]))

## Display the Combined Anomaly Data Time Series for these *ResultType*s
Select the event type(s) to view above and run the following cell.

> Warning: selecting multiple types make produce unclear trend an anomalies

In [ ]:
from msticpy.nbtools.timeseries import display_timeseries_anomolies

event_types = selected_event_types()
ts_data = signin_ts_df[signin_ts_df["ResultType"].isin(event_types)]
if not ts_data.empty:
    if len(event_types) <= 2:
        title = [
            f"{eid} - {desc}"
            for eid, desc in result_types.items()
            if str(eid) in event_types
        ]
    else:
        title = event_types
    plot = display_timeseries_anomolies(
        data=ts_data, title=f"Timeline anomalies for event types {title}", height=800
    )
else:
    md("No events matching selected types")

## Extract time-intervals around the anomalies
Next we want to narrow down the time ranges to the periods of greatest anomaly activity.

> Note: the selection of events is based on the last graph.<br>

First, we will extract time periods around each positive anomaly and calculate a 1 hour
time window around it (merging successive anomaly points into single time regions).

In [ ]:
from msticpy.analysis.timeseries import (
    create_time_period_kqlfilter,
    extract_anomaly_periods,
)

anom_periods = extract_anomaly_periods(
    signin_ts_df[signin_ts_df["ResultType"].isin(selected_event_types())], period="1H"
)
anom_per_df = pd.DataFrame(data=anom_periods.items(), columns=["start", "end"])
md(f"Number of anomalous periods extracted = {len(anom_per_df)}")

all_periods_crit = (signin_ts_df["ResultType"].isin(selected_event_types())) & (
    signin_ts_df["anomalies"] == 1
)
for idx, row in anom_per_df.iterrows():
    per_crit = (signin_ts_df["TimeGenerated"] >= row.start) & (
        signin_ts_df["TimeGenerated"] <= row.end
    )
    anom_count = signin_ts_df[all_periods_crit & per_crit].anomalies.count()
    anom_per_df.loc[idx, "anom_count"] = anom_count

### Show extracted periods graphically
#### (just a bit easier than eyeballing a list)

Usually the graph below will show a few large blocks and a lot of smaller periods.

The line thickness is proportional to the number of anomalies in the period.

In [ ]:
from bokeh.io import show
from bokeh.models import Arrow, ColumnDataSource, OpenHead
from bokeh.plotting import figure

avail_periods_plot = figure(
    plot_height=300,
    plot_width=900,
    x_axis_label="Time",
    x_axis_type="datetime",
    x_minor_ticks=10,
    tools=["xwheel_zoom", "box_zoom", "reset", "save", "xpan"],
    toolbar_location="above",
    title="Available time ranges (line thickness == number of anomalies in period)",
)

avail_periods_plot.circle(size=4, x="start", y=1, color="red", source=anom_per_df)
avail_periods_plot.circle(size=4, x="end", y=1, color="red", source=anom_per_df)
for idx, row in anom_per_df.iterrows():
    anoms = row["anom_count"]
    avail_periods_plot.add_layout(
        Arrow(
            end=OpenHead(line_color="green", line_width=1, size=5),
            start=OpenHead(line_color="green", line_width=1, size=5),
            x_start=row["start"],
            y_start=1,
            x_end=row["end"],
            y_end=1,
            line_width=anoms,
        )
    )
show(avail_periods_plot)
md("Zoom in to see period details")

### Select subset of periods to use
We don't want to use all of these anomaly periods - just the ones with most activity

#### Calculate Number of anomalies per period and trim periods that have low numbers of anomalies
We do this to avoid having to query a lot of data.


In [ ]:
anom_thresh = widgets.IntSlider(
    min=1,
    value=10,
    max=anom_per_df.anom_count.max(),
    description="Anomaly threshold",
    layout=widgets.Layout(width="50%"),
    style={"description_width": "150px"},
)
md("Select lower bound for number of anomalies in period.", "bold")
display(anom_thresh)

#### Show the results of applying this threshold

In [ ]:
all_opts = {
    f"{data.start} - {data.end} ({data.end - data.start})  anomalies = {data.anom_count}": idx
    for idx, data in anom_per_df.iterrows()
}
selected = list(anom_per_df[anom_per_df["anom_count"] >= anom_thresh.value].index)
select_periods = widgets.SelectMultiple(
    options=all_opts,
    #     value=selected,
    description="Pick periods to examine",
    layout=widgets.Layout(width="50%", height="300px"),
)
select_periods.value = selected
display(select_periods)

### Plot updated selection ranges to see what we ended up with

In [ ]:
from bokeh.io import show
from bokeh.models import Arrow, ColumnDataSource, OpenHead
from bokeh.plotting import figure

select_periods_plot = figure(
    plot_height=300,
    plot_width=900,
    x_axis_label="Time",
    x_axis_type="datetime",
    x_minor_ticks=10,
    tools=["xwheel_zoom", "box_zoom", "reset", "save", "xpan"],
    toolbar_location="above",
    title="Selected time ranges (thickness == # anomalies)",
)
sel_crit = anom_per_df.index.isin(select_periods.value)

select_periods_plot.circle(
    size=4, x="start", y=1, color="red", source=anom_per_df[sel_crit]
)
select_periods_plot.circle(
    size=4, x="end", y=1, color="red", source=anom_per_df[sel_crit]
)
for idx, row in anom_per_df[sel_crit].iterrows():
    anoms = row["anom_count"]
    select_periods_plot.add_layout(
        Arrow(
            end=OpenHead(line_color="green", line_width=1, size=5),
            start=OpenHead(line_color="green", line_width=1, size=5),
            x_start=row["start"],
            y_start=1,
            x_end=row["end"],
            y_end=1,
            line_width=anoms,
        )
    )
show(select_periods_plot)

<hr>

## 1.3 Utility functions

### 1.3.1 - Caching reference data
To reduce the number of expensive queries we use a caching system.
The notebook **always** queries fresh *failed logon* data. Failure logons
are not affected by this. 

However, in several places we also query reference *success logon* data that
we use for comparison - as baselines indicating normal behavior. 
By default, we will cache this reference data and re-use this
data as long as the date range you are querying for matches the cached data.

> **Note** this has the unfortunate side effect of littering your directory
> with pickled DataFrames but it should be easy to clean these up when
> you no longer need them.

In [ ]:
# File Caching functions
import re
from pathlib import Path


def archive_name(base_name, start, end):
    """Return file name formatted with dates."""
    st_fmt = re.sub(r"[:\.+]", "_", str(start))
    end_fmt = re.sub(r"[:\.+]", "_", str(end))
    return f"{base_name}-{st_fmt}-{end_fmt}.pkl"


def archive_exists(f_name, folder=None):
    """Return True if the filename exists."""
    folder = folder or cache_folder()
    return Path(folder).joinpath(f_name).is_file()


def get_archive(f_name, folder=None):
    """Return the unpickled DataFrame."""
    folder = folder or cache_folder()
    if use_caching() and archive_exists(f_name, folder):
        return pd.read_pickle(str(Path(folder).joinpath(f_name)))
    return None


def save_archive(f_name, data, folder=None):
    """Save (pickle) data to the named file."""
    if use_caching():
        folder = folder or cache_folder()
        data.to_pickle(str(Path(folder).joinpath(f_name)))

### 1.3.2 - Data masking
We import and use a few functions that mask data. You might need to do this
if you are sharing the notebook outside your organization.

The are controlled by the **Mask data** checkbox at the beginning of the notebook.

In [ ]:
from msticpy.data.data_obfus import hash_ip, hash_item, replace_guid


def mask_names(series):
    return series.apply(lambda x: hash_item(x, "@.")) if mask_data() else series


def mask_guids(series):
    return series.apply(lambda x: replace_guid(x)) if mask_data() else series


def mask_ips(series):
    return series.apply(lambda x: hash_ip(x)) if mask_data() else series

### 1.3.3 - Observation Tracking
We'll use this to track findings as we go through the notebook. These are collated into a browseable form at the end of the notebook.

In [ ]:
from collections import namedtuple
from msticpy.nbtools import Observations
obs_log = Observations()

IPEntry = namedtuple("IPEntry", "address, reason, section")
ip_log = []


def fmt_link(title):
    return title.replace(" ", "-")


def create_ip_entries(ip_addresses, reason, section):
    return [IPEntry(address=ip, reason=reason, section=section) for ip in ip_addresses]

## 1.4 Use the selected time periods to get *all* logon failures for those periods

### Fetch all failed and success events and save to 2 dataframes

In [ ]:
from tqdm.auto import tqdm

obs_log.add_observation(caption="Anomaly periods", data=anom_per_df[sel_crit])

periods_dict = {row.start: row.end for row in anom_per_df[sel_crit].itertuples()}
time_filter = create_time_period_kqlfilter(periods_dict)

field_list = [
    "TimeGenerated",
    "DeviceDetail",
    "ResultType",
    "ResultDescription",
    "IPAddress",
    "Location",
    "LocationDetails",
    "AppId",
    "AppDisplayName",
    "UserAgent",
    "UserPrincipalName",
    "UserId",
    "ResourceDisplayName",
    "ClientAppUsed",
]

project_fields = ", ".join(field_list)

qry = """
SigninLogs
| where TimeGenerated between (datetime({start}) .. datetime({end}))
| project {field_list}
| where ResultType {result_filter}
| extend device_str = tostring(DeviceDetail)
"""

md("We split the query into chunks to avoid timeouts")
print("Getting logon failures...")
failed_results = []
for period in tqdm(
    anom_per_df[sel_crit].itertuples(), desc="Queries", total=len(anom_per_df[sel_crit])
):
    failed_results.append(
        qry_prov.exec_query(
            qry.format(
                start=period.start,
                end=period.end,
                field_list=project_fields,
                result_filter=f"in ({selected_event_types(fmt='str')})",
            )
        )
    )
failed_signin_list_df = pd.concat(failed_results, ignore_index=True)


print("Getting sample logon success...")

success_results = []
for idx, period in tqdm(
    enumerate(anom_per_df[sel_crit].itertuples()),
    desc="Queries",
    total=len(anom_per_df[sel_crit]),
):
    file_name = archive_name("success_signin_list_df", period.start, period.end)

    # check if saved data files are present. If so we can use these.
    if not use_caching() or not archive_exists(file_name):
        num_failed_for_time_period = len(failed_results[idx])
        sample_size = max(num_failed_for_time_period, 5000)
        success_df = qry_prov.exec_query(
            qry.format(
                start=period.start,
                end=period.end,
                field_list=project_fields,
                result_filter=f"== 0 | sample {sample_size}",
            )
        )
        save_archive(file_name, data=success_df)  # nop if not using caching
    else:
        success_df = get_archive(file_name)
        print("Using cached ", file_name)
    success_results.append(success_df)

success_signin_list_df = pd.concat(success_results, ignore_index=True)
del failed_results, success_results
print("done")

---
# Part 2 - Finding patterns in failed logons

---

One of the characteristic that we are looking for is repetitive behavior in our failed logons.
Since we can assume that password spray attacks (because of their bulk nature) are carried
out programmatically, we might be able to tell automated logon attempts from legitimate
attempts.

In legitimate logon attempts, we would expect some variability due to things like:
- Authenticating to different services/applications
- Timing of the attempts
- Differences in the IP patterns for the Client IP

We would also expect the logons to originate from a fairly stable set of locations.

Finally, we can check characteristics of the client device used and see if it matches any used by legitimate successful logons.

Features
- 2.1 Properties with low variability
- 2.2 IP Block usage
- 2.3 Login Request timing
- 2.4 IP Location
- 2.5 User Agent strings
- 2.6 IP Addresses with multiple users

## 2.1 Looking for Properties that have a low variability in Failed vs. Success Logons

### Select the *ResultType* subset to use in this section

Multiple selections are possible but you may find the results are clearer in this section if you work on one failure type at a time.

Select the Failure type(s) and run the rest of the cells in this section. Then repeat for different types.

> Note This selection only affects the subsequent queries in this section.

In [ ]:
opts = {
    f"{res[0]}: {res[1]}": res[0]
    for res in failed_signin_list_df[["ResultType", "ResultDescription"]]
    .drop_duplicates()
    .values
}
select_res_type = widgets.SelectMultiple(
    options=opts,
    description="Select failure type:",
    layout=widgets.Layout(width="70%", height="150px"),
    style={"description_width": "initial"},
)
select_res_type.value = list(opts.values())
select_res_type

### Display subset (top 30)
Typically, you will see logon patterns with high numbers of failures near the top of the list

In [ ]:
disp_cols = ["ResourceDisplayName", "AppId", "ResultType", "device_str", "UserAgent"]
fail_res_type_filter = failed_signin_list_df["ResultType"].isin(select_res_type.value)
failed_subset = (
    failed_signin_list_df[[*disp_cols, "TimeGenerated"]][fail_res_type_filter]
    .groupby(disp_cols)
    .count()
    .rename(columns={"TimeGenerated": "Count"})
    .query("Count > 5")
    .sort_values("Count", ascending=False)
)

display(failed_subset.head(15))

### Select the top N rows to examine

In [ ]:
def_value = len(failed_subset) // 5 if len(failed_subset) > 30 else len(failed_subset)
rows_select = widgets.IntSlider(
    min=1,
    max=len(failed_subset),
    value=def_value,
    description="Top N rows",
    layout=widgets.Layout(width="60%"),
)
rows_select

## Comparing Variability of property values

In the first chart we can see which properties have relatively little variability (near zero)
and properties that have different values for many or most failed requests. 

We can note that the IP address field shows a lot of variability for our failed attempts.
It is common for password spray attacks to cycle through addresses to try to avoid
simple brute force detections.

In the second chart we are plotting the relative variability of failed vs. success events.
If the properties have about the same level of variability in failed and success the 
score in the second chart will be close to 1.

Some properties where we often see significantly less variability in failed vs. success
logons:

- device_str (this is a flattened version of the dynamic JSON field DeviceDetails). It
  contains data from the user agent (UA) such as OS version and type.
- UserAgent
- ResourceDisplayName
- AppID and AppDisplayName - there is a one-to-one correspondence between these two,
  so these are essentially the same single property.
 

In [ ]:
failed_subset_df = failed_subset.reset_index().iloc[: rows_select.value]
cols = list(failed_subset_df.columns)
cols.remove("Count")
failed_check_vars = failed_signin_list_df.merge(failed_subset_df, on=cols)


def get_unique_prop(ser):
    """Return the relative uniqueness of values for each property."""
    try:
        return len(ser.unique()) / len(ser)
    except TypeError:
        return np.nan


# Calculate the relative variability of success vs. failure properties
auth_variability_df = (
    pd.DataFrame(
        data=[
            failed_check_vars.drop(columns=["TimeGenerated"]).apply(
                lambda x: get_unique_prop(x)
            ),
            success_signin_list_df.drop(columns=["TimeGenerated"]).apply(
                lambda x: get_unique_prop(x)
            ),
        ],
        index=["failed", "success"],
    )
    .dropna(axis=1)
    .T
)

del failed_check_vars

# Plot a bar graph showing variability in failed logons
failed_feat_props_fig = (auth_variability_df[["failed"]]).hvplot.barh(
    title="Properties - variability in failed logons",
    rot=90,
    height=400,
    width=600,
    color="red",
)
success_feat_props_fig = (auth_variability_df[["success"]]).hvplot.barh(
    title="Properties - variability in success logons",
    rot=90,
    height=400,
    width=600,
)

# Plot the ratio of variability in success vs. failed
md(
    "The columns in the second plot with a high value show repetition not typical of successful logons",
    "large",
)
comp_feat_props_fig = (
    auth_variability_df["success"] / auth_variability_df["failed"]
).hvplot.barh(
    title="Properties - ratio of variability in success / failed.",
    rot=90,
    height=400,
    width=600,
    color="orange",
)
display(failed_feat_props_fig + success_feat_props_fig + comp_feat_props_fig)
md(
    """
<br>
<b>Ratio of property variability for success/failed events</b><br>
Numbers > 1 means that there is more variability in successful logins than
failed attempts
"""
)
display(
    pd.DataFrame(
        auth_variability_df["success"] / auth_variability_df["failed"],
        columns=["Ratio"],
    )
)

wrn_not_enough_repeats = """
<p style='background-color: gold; font-size: medium'>
Not enough properties with repetitive values for this section to work.
Try with a different subset of input events (fewer ResultTypes)
</p>
"""
if (
    len(
        auth_variability_df[
            (auth_variability_df["success"] / auth_variability_df["failed"]) > 2
        ]
    )
    < 3
):
    md(wrn_not_enough_repeats)

## Observations

We may see that the failed requests repeated several of the request parameters.
These commonly include:
- UserPrincipalName
- UserAgent
- AppId
- IPAddress

Lots of repetition in failed requests points to logons caming from an automated source.

> **Note**: the "automated source" may not be an attacker - it could
> just be some forgotten device or service.

### Sample of failure logons with repeating properties

In [ ]:
# Get the columns with biggest delta
delta_threshold = 2.0
repeating_cols = list(
    auth_variability_df[
        (auth_variability_df["success"] / auth_variability_df["failed"])
        > delta_threshold
    ].index.values
)


# Group by these columns to find the repeating values
repeating_fail_props = (
    failed_signin_list_df[["TimeGenerated", *repeating_cols]]
    .groupby(repeating_cols)
    .count()
    .reset_index()
    .rename(columns={"TimeGenerated": "Count"})
)
# if "UserPrincipalName" in repeating_cols:
#     repeating_fail_props = repeating_fail_props.assign(UserPrincipalName=lambda x: mask_names(x.UserPrincipalName))
# if "UserId" in repeating_cols:
#     repeating_fail_props = repeating_fail_props.assign(UserId=lambda x: mask_guids(x.UserId))

md(
    "Top 20 Login failure patterns matching our 'Low Variability' property values.",
    "bold",
)
rpt_fail_cols = list(repeating_fail_props.columns)
rpt_fail_cols.remove("Count")
obs_data = (
    repeating_fail_props
    .assign(UserPrincipalName=lambda x: mask_names(x.UserPrincipalName))
    .assign(UserId=lambda x: mask_guids(x.UserId))
    .groupby(rpt_fail_cols)
    .sum()
    .sort_values("Count", ascending=False)
    .head(20)
)
display(obs_data)
obs_log.add_observation(
    caption="Sample of failure logons with repeating properties",
    description="Top 20 Login failure patterns matching our 'Low Variability' property values.",
    link=fmt_link("Sample of failure logons with repeating properties"),
    data=obs_data,
    score=1 if len(obs_data) < 5 else 2,
)

### Set Logon Count Threshold for Searching for successful logons with similar properties
We use the 90th percentile by default

In [ ]:
# Set a threshold over which we want to search for similar patterns
md(
    "Set theshold for repeating values - search for patterns with Count >= threshold",
    "bold",
)

repeat_threshold = widgets.BoundedIntText(
    value=int(repeating_fail_props.Count.quantile(q=0.9)),
    min=0,
    max=repeating_fail_props.Count.max(),
    step=1,
    description="Threshold",
    disabled=False,
)

display(repeat_threshold)
md(
    "If the following query fails/times out increase this threshold (default = 90th percentile)"
)

## Use suspicious logon properties to check for other activity

We can use the values that we see in the anomalous failures to look for other logons using the same pattern

In [ ]:
# Build filter list from repeating column values
from datetime import timedelta

# add IPs from previous query >= threshold to ip_log
ips_to_log = (
    repeating_fail_props.merge(failed_signin_list_df, on=repeating_cols)
    .query("Count >= 51")
    .IPAddress.unique()
)
ip_log.extend(
    create_ip_entries(
        ips_to_log,
        reason="Failed logons with repeating properties",
        section="Sample of failure logons with repeating properties",
    )
)

freq_fail_props = repeating_fail_props[
    repeating_fail_props["Count"] >= repeat_threshold.value
]

suspect_signins_filt = []
for _, row in freq_fail_props[repeating_cols].drop_duplicates().iterrows():
    # for each row build a filter experssion combining each
    # column name and value with "AND"
    suspect_signins_filt.append(
        " and ".join((f"{col}=='{row[col]}'" for col in repeating_cols))
    )

# Join the fitler list with "OR"
where_suspect = " or ".join(f"({filt_item})" for filt_item in suspect_signins_filt)
where_suspect
# Search from start of anomalies + 5 days
start = failed_signin_list_df.TimeGenerated.min()
end = start + timedelta(5)

# Create and run the query
qry = """
SigninLogs
| project {field_list}
| where TimeGenerated >= datetime({start}) and TimeGenerated <= datetime({end})
| extend device_str = tostring(DeviceDetail)
| where {suspect_filter}
| where ResultType == 0
"""
print("Querying for success logins with matching properties...")
logins_to_investigate = qry_prov.exec_query(
    qry.format(
        suspect_filter=where_suspect, start=start, end=end, field_list=project_fields
    )
)
print("done")

## Successful logons with suspicious login properties
Our results from the query show some successful logons (which we should investigate)
The majority though, are different types of failures - some even blocked because of
a known malicious IP source

In [ ]:
group_cols = [
    "ResourceDisplayName",
    "AppDisplayName",
    "UserAgent",
    "ResultType",
    "ResultDescription",
]

if logins_to_investigate.empty:
    md("No successful logins found with matching properties.")
else:
    md(
        "<p style='background-color: coral; font-size: large'>Successful logins found with matching properties.</p>"
    )
    obs_data = (
        logins_to_investigate[["Location", *group_cols]]
        .groupby(group_cols)
        .agg(
            logon_count=pd.NamedAgg("Location", "count"),
            location_count=pd.NamedAgg("Location", pd.Series.nunique),
        )
    )
    display(obs_data)

In [ ]:
warning_mssg = """
<hr>
<p style='background-color: coral; font-size: large; margin: 5pt'>These logins should be investigated.</p>
<hr>
"""
if not logins_to_investigate.empty:
    md(warning_mssg)
    if "UserPrincipalName" in group_cols:
        logins_to_investigate = logins_to_investigate.assign(
            UserPrincipalName=lambda x: mask_names(x.UserPrincipalName)
        )
    if "UserId" in group_cols:
        logins_to_investigate = logins_to_investigate.assign(
            UserId=lambda x: mask_guids(x.UserId)
        )
    obs_data = logins_to_investigate.groupby(
        ["UserPrincipalName", "UserAgent", "AppDisplayName"]
    ).agg(
        logon_count=pd.NamedAgg("Location", "count"),
        location_count=pd.NamedAgg("Location", pd.Series.nunique),
        start=pd.NamedAgg("TimeGenerated", "min"),
        end=pd.NamedAgg("TimeGenerated", "max"),
    )
    display(obs_data)
    obs_log.add_observation(
        caption="*Successful logons with suspicious login properties",
        description="Success logins matching our 'Low Variability' property values.",
        data=obs_data,
        link=fmt_link("Successful logons with suspicious login properties"),
        score=5,
    )
    ip_log.extend(
        create_ip_entries(
            ip_addresses=logins_to_investigate.IPAddress.unique(),
            reason="Successful login with suspicious repeated properties",
            section="Successful logons with suspicious login properties",
        )
    )

### Office Activity from suspicious IP Addresses

In [ ]:
# Add any IPs from successful logins
suspicious_ips = list(logins_to_investigate.IPAddress.unique())
# Add the IPs from our original failure list
if "IPAddress" in freq_fail_props:
    suspicious_ips.extend(list(freq_fail_props.IPAddress.unique()))

suspicious_ips_list = ", ".join([f'"{ip}"' for ip in suspicious_ips])
if suspicious_ips_list:

    office_activity = qry_prov.Office365.list_activity_for_ip(
        start=start, end=end, ip_address_list=suspicious_ips, split_query_by="1d"
    )
    if not office_activity.empty:
        md(
            "<p style='background-color: gold' >Consider investigating the following activity</p>"
        )
    obs_data = (
        office_activity[
            ["TimeGenerated", "OfficeWorkload", "Operation", "UserId", "ClientIP"]
        ]
        .assign(UserId=lambda x: mask_names(x.UserId))
        .groupby(["UserId", "ClientIP", "OfficeWorkload"])
        .agg(
            events=pd.NamedAgg("TimeGenerated", "count"),
            start=pd.NamedAgg("TimeGenerated", "min"),
            end=pd.NamedAgg("TimeGenerated", "max"),
            operations=pd.NamedAgg(
                "Operation", aggfunc=lambda x: sorted(x.unique().tolist())
            ),
        )
    )
    display(obs_data)
    obs_log.add_observation(
        caption="Office Activity from suspicious IP Addresses",
        description="Office activity with IPs matching our 'Low Variability' property values.",
        data=obs_data,
        link=fmt_link("Office Activity from suspicious IP Addresses"),
        score=5,
    )
    ip_log.extend(
        create_ip_entries(
            ip_addresses=office_activity.ClientIP.unique(),
            reason="Successful login with suspicious repeated properties",
            section="Successful logons with suspicious login properties",
        )
    )

In [ ]:
del office_activity

## Use suspicious logon properties to check for other activity 
### (excluding Username/ID/IPAddress)

We can use the values that we see in the anomalous failures to look for other logons using the same pattern

In [ ]:
# Build filter list from repeating column values
from datetime import timedelta

excluded_cols = ["UserPrincipalName", "UserId", "IPAddress"]
freq_fail_props = repeating_fail_props[
    repeating_fail_props["Count"] > repeat_threshold.value
]

no_user_cols = list(set(repeating_cols) - set(excluded_cols))
no_user_cols
suspect_signins_filt = []
for _, row in freq_fail_props[no_user_cols].drop_duplicates().iterrows():
    # for each row build a filter experssion combining each
    # column name and value with "AND"
    suspect_signins_filt.append(
        " and ".join((f"{col}=='{row[col]}'" for col in no_user_cols))
    )

# Join the filter list with "OR"
where_suspect = " or ".join(f"({filt_item})" for filt_item in suspect_signins_filt)

# Search from start of anomalies + 5 days
start = failed_signin_list_df.TimeGenerated.min()
end = start + timedelta(5)
# Create and run the query
qry = """
SigninLogs
| project {field_list}
| where TimeGenerated >= datetime({start}) and TimeGenerated <= datetime({end})
| extend device_str = tostring(DeviceDetail)
| where {suspect_filter}
| where ResultType == 0
"""
logins_to_investigate_no_user = None
try:
    logins_to_investigate_no_user = qry_prov.exec_query(
        qry.format(
            suspect_filter=where_suspect,
            start=start,
            end=end,
            field_list=project_fields,
        )
    )

except:
    md_warn("Query failed - try with higher logon threshold (above)")

if (
    logins_to_investigate_no_user is not None
    and not logins_to_investigate_no_user.empty
):
    group_cols = [
        "ResourceDisplayName",
        "AppDisplayName",
        "UserAgent",
        "ResultType",
        "ResultDescription",
    ]

    obs_data = (
        logins_to_investigate_no_user.assign(
            UserPrincipalName=lambda x: mask_names(x.UserPrincipalName),
            UserId=lambda x: mask_guids(x.UserId),
            IPAddress=lambda x: mask_ips(x.IPAddress)
        )
        .groupby([*group_cols, *excluded_cols])
        .agg(
            logon_count=pd.NamedAgg("Location", "count"),
            location_count=pd.NamedAgg("Location", pd.Series.nunique),
        )
    )
    md(warning_mssg)
    display(obs_data)
    obs_log.add_observation(
        caption="*Successful logons from sources using suspicious login properties",
        description="Similar to earlier query but excluding matching on name or IP.",
        data=obs_data,
        link=fmt_link("Use suspicious logon properties to check for other activity"),
        score=3,
    )
    ip_log.extend(
        create_ip_entries(
            ip_addresses=logins_to_investigate_no_user.IPAddress.unique(),
            reason="Successful login with suspicious repeated properties",
            section="Use suspicious logon properties to check for other activity",
        )
    )

## Observations
Properties with a comparative variablility score close to 1 mean that failed requests have little to distinguish them from successful requests.
Properties with much higher scores mean that legitimate requests are much more variable.

The columns with repetitious values in the **failed logons** data are:
- AppDisplayName
- AppId (1:1 correspondence with AppDisplayName
- ResourceDisplayName
- device_str

We can use requests with repeated values for the properties to spot potential password spray and other brute-force attacks.
```
    SigninLogs
    | extend device_str = tostring(DeviceDetail)
    | where (susp_col1 == "susp_val1" and "col2" == "susp_val2" ....)
        or (susp_col1 == "susp_val1" and "col2" == "susp_val2" ....)
```


## 2.x.1 Looking at the time difference between login requests

> **Note** although potentially interesting this item isn't used in further analysis in the notebook.

We'd expect automated attacks to show short and regular intervals between requests. Even randomly delayed/spaced requests will likely exhibit some pattern over time.

We may see some difference in the timing between successive events

### Time Separation between Failed Logons vs Success Logons 

Plot of a histograms of the delay between requests. 
There is a strong peak
show that many of the failed attempts occured over a short period of time.

In [ ]:
time_diffs_f = (
    failed_signin_list_df.sort_values("TimeGenerated")["TimeGenerated"]
    .diff(1)
    .dt.microseconds.dropna()
)
time_diffs_s = (
    success_signin_list_df.sort_values("TimeGenerated")["TimeGenerated"]
    .diff(1)
    .dt.microseconds.dropna()
)
(
    time_diffs_f[time_diffs_f < 1000000000].hvplot.hist(
        bins=np.linspace(0, 1000000, 35),
        rot=90,
        title="Failed logons",
        height=400,
        width=400,
        color="orange",
    )
    + time_diffs_s[time_diffs_s < 1000000000].hvplot.hist(
        bins=np.linspace(0, 1000000, 35),
        rot=90,
        title="Success logons",
        height=400,
        width=400,
    )
)

## 2.x.2 Use of Consecutive or Closely Clustered IP addresses

> **Note** although potentially interesting this item isn't used in further analysis in the notebook.

We noted earlier that password spray attackers tend to change the source IP address
after one or a small number of requests.
Something we can look for is whether the numerical spacing between the failing IP addresses
is noticably different to that that we find for legitimate users. 

To do this we need to 
- convert the IP addresses into numbers (the 32-bit integer
  equivalent of the IP Address). 
- calculate the numerical difference
  between the IP addresses in successive events using pandas.diff() function.
  This will show us how much successive requests are differing.

You would expect this difference between successive requests
to be fairly large and random if the sources are users logging in from offices, home,
customer sites, hotels, etc. If an automated attack is randomly picking IP addresses
from small pools of available IPs it might look more homogeneous.

In [ ]:
from ipaddress import ip_address

ip_cols = ["TimeGenerated", "IPAddress"]
failed_ips = failed_signin_list_df[ip_cols].copy()
success_ips = success_signin_list_df.copy()

md(
    """
    If IPs for failed logons are unique or very similar,
    most will be clustered near 0
""",
    "bold",
)
display(
    np.log10(failed_ips["IPAddress"].value_counts()).hvplot.hist(
        title="(log10) Number of times IP address are re-used in failed and success logins",
        bins=100,
        color="orange",
    )
    + np.log10(success_ips["IPAddress"].value_counts()).hvplot.hist(
        bins=100,
        color="blue",
    )
)

failed_ips["ip_num"] = failed_ips["IPAddress"].apply(
    lambda x: int.from_bytes(ip_address(x).packed, byteorder="big")
)

# Plot timeline of numerical differences between consecutive IPs
clip_val = 10 ** 20
failed_ip_diff_df = pd.concat(
    [
        failed_ips.sort_values("TimeGenerated")["TimeGenerated"],
        failed_ips.sort_values("TimeGenerated")["ip_num"]
        .diff(1)
        .fillna(0)
        .astype("float64"),
    ],
    axis=1,
).dropna()
failed_ip_diff_df.loc[failed_ip_diff_df["ip_num"] != 0, "ip_num"] = np.log10(
    np.abs(failed_ip_diff_df[failed_ip_diff_df["ip_num"] != 0].ip_num)
).fillna(0)
success_ips = success_signin_list_df.copy()

# print("Some IP address are repeated but most are unique.")
# print(success_ips.sort_values("IPAddress")["IPAddress"].value_counts())
# Convert IP Address to integers
success_ips["ip_num"] = success_ips["IPAddress"].apply(
    lambda x: int.from_bytes(ip_address(x).packed[:3], byteorder="big")
)

# Plot timeline of numerical differences between consecutive IPs
success_ip_diff_df = pd.concat(
    [
        success_ips.sort_values("TimeGenerated")["TimeGenerated"],
        success_ips.sort_values("TimeGenerated")["ip_num"]
        .diff(1)
        .fillna(0)
        .astype("float64"),
    ],
    axis=1,
).dropna()
success_ip_diff_df.loc[success_ip_diff_df["ip_num"] != 0, "ip_num"] = np.log10(
    np.abs(success_ip_diff_df[success_ip_diff_df["ip_num"] != 0].ip_num)
).fillna(0)

# Plot the graphs
md(
    """
    These time graphs show the numeric differences between successive
    IP addresses for failed (orange) and success (blue) logins
    """,
    "bold",
)
display(
    failed_ip_diff_df.hvplot.hist(
        bins=500, line_color="orange", title="Histogram of IP differences"
    )
    * success_ip_diff_df.hvplot.hist(bins=500, line_color="blue")
)
md(
    """
    Timeline of  IP numeric differences
    (y axis = IP numeric value)
    """,
    "bold",
)
display(
    failed_ip_diff_df.hvplot.scatter(
        title="Timeline - Success/Failed logon IP diff between successive attempts",
        x="TimeGenerated",
        color="orange",
        size=3,
    )
    * success_ip_diff_df.hvplot.scatter(
        x="TimeGenerated", color="blue", size=1, alpha=0.25
    )
)
del success_ips, failed_ips

### Observations:
- Failed logons are generally come from a consistent IP space but show intermittent IP sources
  that have very different values.
- Success logons tend to be clustered more homogenously
- However, the distribution of the vast majority of login failure IPs looks very simlar
  to the success logins
- While an interesting observation, it's difficult to picture how to use this data in further analysis

### A formula

Although obvious from the graphic, it would be nice to see this numerically.
We can use the standard deviation of the suspect IP address.

If the origin of the suspect IP addresses is users forgetting and retry their
passwords the variance of these divided by the variance of the successful
logins should be close to one.

In [ ]:
std_fail = failed_ip_diff_df.ip_num.std()
std_success = success_ip_diff_df.ip_num.std()
md(f"Stddev of failure IPs is {std_fail:f}")
md(f"Stddev of success IPs is {std_success:f}")
md(f"Ratio of fail/success stddev IPs is {std_fail/std_success:f}", "bold")
md(
    "(expected is close to 1.0 if failures followed the same pattern as success source IPs)"
)

---
## 2.2 Comparing IP Blocks for Failed and Success Logons

We can reuse our integer representation of the IP Addresses to do 
some simple scatter plots. In this case though, we "aggregate" the IP
Addresses into blocks by only using the upper 3 bytes of the 4 byte address.
So 192.168.1.23 and 192.168.1.55 would both appear in the 192.168.1.0 block.

The data below shows that there are
distinct blocks of failed logon source IP addresses that are very different
to the majority of success logons. If these were genuine logon failures from 
users we would expect to see the failures occupying the approximately
the same address space as the successful logons.

> **NOTE** the scaling of the graph suggests that many of the failing
> IP addresses are using the same netblock as successful IPs but if you use
> the graph zoom controls to zoom in to particular areas you will
> usually see that they do not overlap.


In [ ]:
# We need to get a representative sample of org's normal IP usage
# pick a day near end of selection period - pick Wed to avoid Monday holidays and varying weekends (e.g. Israel)
start_rng = pd.Timestamp(query_time.end) - pd.Timedelta("7d")
wednesday = [
    day for day in pd.date_range(start_rng, periods=7, freq="1D") if day.dayofweek == 2
][0]
thursday = [
    day for day in pd.date_range(start_rng, periods=7, freq="1D") if day.dayofweek == 3
][0]


# Get summary statistics for IP block usage
print("Querying success login IP statistics")
qry = f"""
SigninLogs
| where TimeGenerated > datetime({wednesday}) and TimeGenerated < datetime({thursday})
| where ResultType == 0
| extend IPBlock_str = iff(
    IPAddress contains("."), 
    strcat_array(array_slice(split(IPAddress, "."), 0, 2), "."), // IPv4 and v6/v4
    strcat_array(array_slice(split(IPAddress, ":"), 0, 3), ":")
    )
| summarize Count = count() by IPBlock_str
"""

file_name = archive_name("ip_block_usage", wednesday, thursday)
if not use_caching() or not archive_exists(file_name):
    print("querying data...")
    ip_block_usage = qry_prov.exec_query(qry)
    save_archive(file_name, ip_block_usage)
else:
    ip_block_usage = get_archive(file_name)
print("done")


# Calculate ip block usage figure
def calc_ipblock_usage(ip_df):
    """Give a logarithmic score to number of times block is used."""
    ip_df["usage"] = np.floor(np.log(ip_df["Count"])) + 1


print("Calculating IP Block useage for success logins")
calc_ipblock_usage(ip_block_usage)


# Merge with Success logon usage to get netblock usage for the failed logon attempts
failed_signin_list_df["IPBlock_str"] = failed_signin_list_df.IPAddress.str.extract(
    "(\d+\.\d+.\d+)", expand=False
).fillna("1.1.1")
failed_signin_usage = failed_signin_list_df.merge(
    ip_block_usage[["IPBlock_str", "usage"]], on="IPBlock_str", how="left"
)
failed_signin_usage.fillna(0, inplace=True)
print("done")

md("<br>Usage Scoring", "bold")
md(
    "The 'usage' score is the log10 of the number of successful logons for an IP Block.<br>"
)

bins = (ip_block_usage.usage.nunique() + 1) * 2
display(
    failed_signin_usage["usage"].hvplot.hist(
        bins=bins,
        color="orange",
        title="Number of failed logins by known IP block usage",
    )
    + ip_block_usage["usage"].hvplot.hist(
        bins=bins,
        line_color="blue",
        title="Number of success logins by known IP block usage",
    )
)

### Failed logons from IP addresses (blocks) not used by Organization (Top 30)

In [ ]:
signin_data = (
    failed_signin_usage[failed_signin_usage["usage"] == 0]
    .groupby([*group_cols, "IPAddress"])
    .agg(
        logon_count=pd.NamedAgg("Location", "count"),
        location_count=pd.NamedAgg("Location", "nunique"),
    )
    .sort_values("logon_count", ascending=False)
    .head(30)
)

# We're resetting and setting the index again so that we can
# mask the IP data
obs_data = (
    signin_data.reset_index()
    .assign(IPAddress=lambda x: mask_ips(x.IPAddress))
    .groupby([*group_cols, "IPAddress"])
    .agg({"logon_count": "sum", "location_count": "sum"})
    .sort_values("logon_count", ascending=False)
    .head(30)
)
display(obs_data)
obs_log.add_observation(
    caption="Failed logons from IPs blocks not used by organization",
    data=obs_data,
    link=fmt_link(
        "Failed logons from IP addresses (blocks) not used by Organization (Top 30)"
    ),
    score=2,
)
ip_log.extend(
    create_ip_entries(
        ip_addresses=signin_data.reset_index().IPAddress.unique(),
        reason="Failed logons from IPs blocks not used by organization",
        section="Failed logons from IP addresses (blocks) not used by Organization (Top 30)",
    )
)

### Failed logons from IP blocks rarely used by organization (Top 30)

In [ ]:
signin_data = (
    failed_signin_usage[failed_signin_usage["usage"].isin((1, 2))]
    .groupby([*group_cols, "IPBlock_str", "IPAddress"])
    .agg(
        logon_count=pd.NamedAgg("Location", "count"),
        location_count=pd.NamedAgg("Location", "nunique"),
    )
    .sort_values("logon_count", ascending=False)
    .head(30)
)
# We're resetting and setting the index again so that we can
# mask the IP data
obs_data = (
    signin_data.reset_index()
    .assign(IPAddress=lambda x: mask_ips(x.IPAddress))
    .groupby([*group_cols, "IPAddress"])
    .agg({"logon_count": "sum", "location_count": "sum"})
    .sort_values("logon_count", ascending=False)
    .head(30)
)
display(obs_data)
obs_log.add_observation(
    caption="Failed logons from IPs blocks rarely used (score of 1 or 2) by organization",
    data=obs_data,
    link=fmt_link("Failed logons from IP blocks rarely used by organization (Top 30)"),
    score=2,
)
ip_log.extend(
    create_ip_entries(
        ip_addresses=signin_data.reset_index().IPAddress.unique(),
        reason="Failed logons from IPs blocks rarely used by organization",
        section="Failed logons from IP blocks rarely used by organization (Top 30)",
    )
)

### Failed logons from most commonly used IP blocks (Top 30)

In [ ]:
signin_data = (
    failed_signin_usage[failed_signin_usage["usage"] > 2]
    .groupby([*group_cols, "IPBlock_str", "IPAddress"])
    .agg(
        logon_count=pd.NamedAgg("Location", "count"),
        location_count=pd.NamedAgg("Location", "nunique"),
        mean_usage=pd.NamedAgg("usage", "mean"),
    )
    .sort_values("logon_count", ascending=False)
    .head(30)
)
# We're resetting and setting the index again so that we can
# mask the IP data
obs_data = (
    signin_data.reset_index()
    .assign(IPAddress=lambda x: mask_ips(x.IPAddress))
    .groupby([*group_cols, "IPAddress"])
    .agg({"logon_count": "sum", "location_count": "sum"})
    .sort_values("logon_count", ascending=False)
    .head(30)
)
display(obs_data)
obs_log.add_observation(
    caption="Failed logons from IPs blocks commonly used by organization",
    data=obs_data,
    link=fmt_link("Failed logons from most commonly used IP blocks (Top 30)"),
    score=1,
)
ip_log.extend(
    create_ip_entries(
        ip_addresses=signin_data.reset_index().query("logon_count > 10").IPAddress.unique(),
        reason="Failed logons from IPs blocks commonly used by organization",
        section="Failed logons from most commonly used IP blocks (Top 30)",
    )
)

### Timeplot for failed logon IP Blocks (showing usage relative to success logons)
Use the legend to deselect usage/ResultType to focus on the remaining logon failures.
> Note deselecting doesn't hide the deleselect points, it just makes them more transparent<br>
> When many points are overlaid at the same location the color will strengthen

If you have a lot of data to display the graphics can be difficult to interpret.
Use the resultType selection list to display subsets

In [ ]:
select_res_type.value = list(select_res_type.options.values())
select_res_type

In [ ]:
fail_res_type_filter = failed_signin_usage["ResultType"].isin(select_res_type.value)


def ip_blk_to_num(ip_block):
    try:
        nums = ip_block.split(".")
        return int(nums[0]) * 65536 + int(nums[1]) * 256 + int(nums[2])
    except AttributeError:
        print(ip_block)


failed_signin_usage["Block_num"] = failed_signin_usage.apply(
    lambda x: ip_blk_to_num(x.IPBlock_str), axis=1
)

failed_signin_usage["TG_num"] = pd.to_numeric(failed_signin_usage["TimeGenerated"])
ip_blk_usage = failed_signin_usage[fail_res_type_filter].hvplot.scatter(
    x="TG_num",
    y="Block_num",
    by="usage",
    grid=True,
    title="IP Timeplot for Failed Logons (key=freq of use in success logins)",
    height=600,
    width=600,
    hover_cols=[
        "Location",
        "ResultDescription",
        "IPAddress",
        "UserPrincipalName",
        "IPAddress",
    ],
    xlabel="time",
    size=8,
    muted_alpha=0,
)


ip_blk_res_type = failed_signin_usage[fail_res_type_filter].hvplot.scatter(
    x="TG_num",
    y="Block_num",
    by="ResultType",
    grid=True,
    title="IP Timeplot for Failed Logons (key=FailureType)",
    height=600,
    width=600,
    hover_cols=[
        "Location",
        "ResultDescription",
        "IPAddress",
        "UserPrincipalName",
        "IPAddress",
    ],
    xlabel="time",
    size=8,
    muted_alpha=0,
)


display(ip_blk_usage + ip_blk_res_type)
md("ResultType Key", "bold")
for res_type in sorted(list(failed_signin_usage["ResultType"].unique())):
    md(f"{res_type:<10} {result_types[res_type]}".replace(" ", "&nbsp;"))

### Record IP Block usage for failed login

In [ ]:
failed_signin_list_df["known_ip_block"] = failed_signin_usage["usage"]
del failed_signin_usage

### Observations
- Failed logons from familiar netblocks tend to be consistent - i.e. it looks as though they are always originating from the same place
- There is an overlap in failure types - logins from familiar netblocks tend to be 50126
- Netblock origin seems to be a good predictor of an suspicious/failed login

## 2.3 Login Request Timing
Can we identify patterns based on timing of logon failures?

The goal is to use timing patterns to ID attackers distributed across source IP addresses but using regularly-scheduled requests.

In [ ]:
display(
    (
        failed_signin_list_df["TimeGenerated"].dt.minute * 60
        + failed_signin_list_df["TimeGenerated"].dt.second
    ).hvplot.hist(
        bins=3600,
        title="Histogram of failed logon requests binned by min/sec past hour",
        line_color="orange",
    )
    + (
        success_signin_list_df["TimeGenerated"].dt.minute * 60
        + success_signin_list_df["TimeGenerated"].dt.second
    ).hvplot.hist(
        bins=3600,
        title="Histogram of success logon requests binned by min/sec past hour",
        line_color="blue",
    )
)

mean_logon_second = len(failed_signin_list_df) / 3600
md(f"Expected count for random distribution is {mean_logon_second:.1f}")
md("Significant peaks above this may indicate scheduled logons")

### Peaks in the above chart indicate multiple login attempts occurring at same minute/second past hour
Since we'd expect automated logon attempts to repeat some properties we can group by:
- AppID
- UserAgent
- Device Properties

In [ ]:
logon_second = (
    failed_signin_list_df["TimeGenerated"].dt.minute * 60
    + failed_signin_list_df["TimeGenerated"].dt.second
).values

# Calculate histogram values
counts, freqs = np.histogram(logon_second, bins=3600)

# Set a threshold over which we want to search for similar patterns
md("Default threshold for count of logins at give min/sec past hour", "bold")
md("default = mean(counts_per_second) + stddev(counts_per_second)")
md("Any events with a count above this threshold will be marked as <b>'scheduled'</b>")
count_threshold = widgets.FloatSlider(
    value=np.std(counts) + mean_logon_second,
    min=0,
    max=counts.max(),
    step=0.1,
    description="Threshold",
    layout=widgets.Layout(width="50%"),
)

del logon_second

display(count_threshold)

In [ ]:
failed_signin_list_df["logon_second"] = (
    failed_signin_list_df["TimeGenerated"].dt.minute * 60
    + failed_signin_list_df["TimeGenerated"].dt.second
)

group_cols = ["AppId", "UserAgent", "device_str", "logon_second"]

# Get the login patterns deemed to be scheduled
scheduled_logins = (
    failed_signin_list_df[["UserPrincipalName", "TimeGenerated", *group_cols]]
    .groupby(group_cols)
    .agg(
        LoginRequests=pd.NamedAgg("TimeGenerated", "count"),
        UniqueUsers=pd.NamedAgg("UserPrincipalName", "nunique"),
    )
    .reset_index()
    .assign(scheduled=True)
    .query("LoginRequests > @count_threshold.value")
)

md("Login patterns determined to be scheduled")
obs_data = (
    scheduled_logins.groupby(["AppId", "UserAgent", "device_str"])
    .agg(
        num_fixed_schedules=pd.NamedAgg("logon_second", "nunique"),
        LoginRequests=pd.NamedAgg("LoginRequests", "sum"),
        UniqueUsers=pd.NamedAgg("UniqueUsers", "sum"),
    )
    .sort_values("LoginRequests", ascending=False)
)
display(obs_data)
obs_log.add_observation(
    caption="Failed logins that appear scheduled (regular time pulse)",
    data=obs_data,
    link=fmt_link("2.3 Login Request Timing"),
    score=1,
)

ip_log.extend(
    create_ip_entries(
        ip_addresses=(
            failed_signin_list_df.merge(
                scheduled_logins, on=group_cols
            ).IPAddress.unique()
        ),
        reason="Failed logons that happen on regular schedule",
        section="2.3 Login Request Timing",
    )
)

# Create a "scheduled" field and assign True for any matching items in
# the scheduled_logins DataFrame
if "scheduled" in failed_signin_list_df.columns:
    failed_signin_list_df.drop(columns="scheduled", inplace=True)
failed_signin_list_df["scheduled"] = (
    failed_signin_list_df[group_cols]
    .merge(scheduled_logins, on=group_cols, how="left")
    .fillna(False)
    .scheduled
)

### Observations
- We found several repeating patterns based on minute + seconds past the hour
- Many of the requests shared the same UserAgent - indicating a common tool and/or source
- These requests could be a faulty or forgotten device or application with an old password.

---
## 2.4 Using IP location data


### Get a sample of successful logins

In [ ]:
qry = f"""
SigninLogs
| where TimeGenerated > datetime({wednesday}) and TimeGenerated < datetime({thursday})
| where ResultType == 0
| extend city = tostring(LocationDetails["city"]),
  countryOrRegion = tostring(LocationDetails["countryOrRegion"]),
  state = tostring(LocationDetails["state"]),
  latitude = toint(LocationDetails["geoCoordinates"]["latitude"]),
  longitude = toint(LocationDetails["geoCoordinates"]["longitude"])
| summarize loc_count=count() by countryOrRegion, state, city, latitude, longitude
"""

print("Querying success login location statistics...")
file_name = archive_name("success_logins_loc", wednesday, thursday)
if not use_caching() or not archive_exists(file_name):
    print("querying sample of current data...")
    success_logins_loc = qry_prov.exec_query(qry)
    save_archive(file_name, success_logins_loc)
else:
    success_logins_loc = get_archive(file_name)
print("done.")

# We need to unpack the Location Details field for failure logins
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    tqdm.pandas(desc="Records")


def expand_locations(data):
    cols = ["TimeGenerated", "IPAddress", "ResultType", "UserAgent"]
    # Note using apply(pd.Series) here is very slow but simple
    print("extracting location details..")
    loc_df = data.progress_apply(
        lambda x: pd.Series(x["LocationDetails"]), axis=1, result_type="expand"
    )
    print("extracting geo coordinates..")
    geo_df = loc_df.progress_apply(
        lambda x: pd.Series(x["geoCoordinates"]), axis=1, result_type="expand"
    )
    comb_loc_df = pd.concat([loc_df, geo_df], axis="columns")
    return pd.concat([data[cols], comb_loc_df], axis="columns")


geo_cols = {
    "city",
    "countryOrRegion",
    "geoCoordinates",
    "state",
    "latitude",
    "longitude",
}
print("Expanding location details to columns.")

# Note try to re-use
print("Processing failed_logins...\n")
failed_logins_loc = expand_locations(failed_signin_list_df)

print("done")

### Viewing the logins spatially
If you squint at the graphics you can kind of make out the shape of some of the continents.

In [ ]:
common_args = dict(x="longitude", y="latitude", height=500, width=900)
display(
    failed_logins_loc.hvplot.scatter(
        **common_args,
        title="Failed vs. Success login locations",
        color="orange",
        by="ResultType",
        alpha=0.8
    )
    * success_logins_loc.hvplot.scatter(**common_args, color="blue", alpha=0.3, size=10)
)
md("Success login locations in green.", "bold")
md(
    "Note: Fainter green indicates fewer logons, more intense green indicates multiple logons."
)

### Show logins with atypical location on a map
This shows all login locations that are not common for successful logins

In [ ]:
def create_ip_entity(row):
    ip_ent = entities.IpAddress(Address=row["IPAddress"])
    ip_ent.AdditionalData["ResultType"] = row["ResultType"]
    ip_ent.AdditionalData["UserAgent"] = row["UserAgent"]
    geo_ent = entities.GeoLocation(
        Latitude=row["latitude"],
        Longitude=row["longitude"],
        City=row["city"],
        State=row["state"],
        CountryName=row["countryOrRegion"],
    )
    ip_ent.Location = geo_ent
    return ip_ent


# get logins to investigate from regions not in
# success logins
other_locs = failed_logins_loc[
    ~(failed_logins_loc["countryOrRegion"].isin(success_logins_loc["countryOrRegion"]))
    & ~(failed_logins_loc["city"].isin(success_logins_loc["city"]))
]
folium = FoliumMap(zoom_start=3)
import warnings

with warnings.catch_warnings():
    folium.add_ip_cluster(
        other_locs.apply(create_ip_entity, axis=1).values, color="orange"
    )

folium.center_map()

display(folium)
obs_log.add_observation(
    caption="Failed logons from IPs locations not used by organization",
    data=folium,
    link=fmt_link("Show logins with atypical location on a map"),
    score=1,
)

## Viewing the numbers of failed logons from atypical locations
We can see that the distribution is not the same but it's pretty difficult
to see what this means. Let's explore the numbers.

In [ ]:
# Calculate probabilities of successful logon from a particlular country
# for both data sets
grouping_cols = ["countryOrRegion", "state", "city"]
login_succ_prob = success_logins_loc[[*grouping_cols, "loc_count"]]
login_succ_prob = (
    login_succ_prob.groupby(grouping_cols)
    .sum()
    .reset_index()
    .rename(columns={"loc_count": "LoginCount"})
)
login_succ_prob["LoginProb"] = (
    login_succ_prob.LoginCount / login_succ_prob.LoginCount.sum()
)

failed_logins_prob = failed_logins_loc[[*grouping_cols, "ResultType"]]
failed_logins_prob = failed_logins_prob.merge(
    login_succ_prob[[*grouping_cols, "LoginProb"]],
    how="left",
    on=["countryOrRegion", "state", "city"],
).fillna(0)
failed_logins_loc_comb = failed_logins_loc
failed_logins_loc_comb["LoginProb"] = failed_logins_prob["LoginProb"]
del failed_logins_prob


md("Statistics for Successful logins", "bold, large")
# Show common and rare locations for Success logons
login_locs1 = (
    login_succ_prob[["countryOrRegion", "state", "city", "LoginProb"]]
    .sort_values("LoginProb", ascending=False)
    .head(10)
    .to_html()
)
login_locs1 = f"<b>Most likely locations for successful logins</b>{login_locs1}"

login_locs2 = (
    login_succ_prob[["countryOrRegion", "state", "city", "LoginProb"]]
    .sort_values("LoginProb", ascending=False)
    .tail(10)
    .to_html()
)
login_locs2 = (
    f"<b>Least likely (non-zero) locations for successful logins</b>{login_locs2}"
)
display(widgets.HBox([widgets.HTML(login_locs1), widgets.HTML(login_locs2)]))

In [ ]:
display(
    failed_logins_loc_comb["LoginProb"].hvplot.hist(
        bins=100,
        title="Probability distribution for failed logins - likelihood that it came from familiar location",
        color="orange",
    )
)

In [ ]:
def col_zero_red(val):
    if val == 0:
        return f"background-color: coral"


md(
    "Mean probabilities that failed logins are coming from familiar locations (zero == unseen location)",
    "bold",
)
obs_data = (
    failed_logins_loc_comb[
        [
            "TimeGenerated",
            "ResultType",
            "UserAgent",
            "countryOrRegion",
            "state",
            "LoginProb",
        ]
    ]
    .groupby(["ResultType", "UserAgent", "countryOrRegion", "state"])
    .agg({"LoginProb": "mean", "TimeGenerated": "count"})
    .rename(
        columns={
            "LoginProb": "LocationLoginProbability",
            "TimeGenerated": "FailedLoginCount",
        }
    )
    .sort_values("FailedLoginCount", ascending=False)
    .head(50)
    .style.background_gradient(
        subset="LocationLoginProbability", cmap="Greens", low=0.0, high=1
    )
    .applymap(col_zero_red, subset=["LocationLoginProbability"])
)
display(obs_data)
obs_log.add_observation(
    caption="Failed logons list by failure count with login location probability",
    data=obs_data,
    link=fmt_link("Viewing the numbers of failed logons from atypical locations"),
    score=2,
)

In [ ]:
md("Top failed logins are coming from locations not seen in success logins", "bold")
obs_data = (
    failed_logins_loc_comb[
        [
            "TimeGenerated",
            "ResultType",
            "UserAgent",
            "countryOrRegion",
            "state",
            "LoginProb",
        ]
    ]
    .groupby(["ResultType", "UserAgent", "countryOrRegion", "state"])
    .agg({"LoginProb": "mean", "TimeGenerated": "count"})
    .rename(
        columns={
            "LoginProb": "LocationLoginProbability",
            "TimeGenerated": "FailedLoginCount",
        }
    )
    .query("LocationLoginProbability == 0")
    .sort_values("FailedLoginCount", ascending=False)
    .head(50)
    .style.background_gradient(
        subset="LocationLoginProbability", cmap="Greens", low=0.0, high=1
    )
    .applymap(col_zero_red, subset=["LocationLoginProbability"])
)
display(obs_data)
obs_log.add_observation(
    caption="Failed logons summary from IPs locations not used by organization",
    data=obs_data,
    link=fmt_link("Viewing the numbers of failed logons from atypical locations"),
    score=2,
)

### Record Familiar Location probability

In [ ]:
failed_signin_list_df["login_loc_prob"] = failed_logins_loc_comb["LoginProb"]
del failed_logins_loc, failed_logins_loc_comb

### Observations
Geographical origin of login requests against a normal pattern of the organization
can help indicate the probability that the request is bogus.
- All categories of failed login came from unlikely locations - with the probability
  an order of magnitude smaller than the least likely legitimate login

However, there are caveats
- the 50125 events for the OneDrive SyncEngine had almost the same probability as
  successful logins coming from a familiar (if rare) geographic location

---
## 2.5 UserAgent strings
UserAgent were recently added to the Azure Active Directory logs. These are potentially more useful
than the DeviceDetails field to identify repetitive requests.
(DeviceDetails was squashed into the device_str field that we used). DeviceDetails contains
slightly different information but is not always populated with anything useful.

We will likely see a similar pattern when comparing UA strings that appear in suspicious data with the
successful logon data.

> **Note** we use both the raw UserAgent string and extract version-agnostic versions<br>
> of UA strings that have all of the numeric information zeroed.

In [ ]:
qry = f"""
SigninLogs
| where TimeGenerated > datetime({wednesday}) and TimeGenerated < datetime({thursday})
| where ResultType == 0
| summarize ua_count=count() by UserAgent
"""
success_user_agents_df = qry_prov.exec_query(qry)

print("Querying User Agent statistics...")
file_name = archive_name("success_user_agents_df", wednesday, thursday)
if not use_caching() or not archive_exists(file_name):
    print("querying sample of current data...")
    success_user_agents_df = qry_prov.exec_query(qry)
    save_archive(file_name, success_user_agents_df)
else:
    success_user_agents_df = get_archive(file_name)
print("done.")

print("Extracting UserAgent version-agnostic patterns.")
print("Calculating usage statistics for User agents and UA pattern.")
# Calculate usage for User agents (calc for both raw UserAgent and UA pattern)
success_user_agents_df = success_user_agents_df.rename(columns={"count": "ua_count"})
success_user_agents_df["usage"] = (
    success_user_agents_df.ua_count / success_user_agents_df.ua_count.sum()
)
success_user_agents_df.head()

# Remove numbers from UserAgent strings to remove version-specific data
success_user_agents_df["ua_pattern"] = success_user_agents_df.UserAgent.str.replace(
    "\d", "x"
)

if "ua_pattern_usage" in success_user_agents_df.columns:
    success_user_agents_df.drop(columns=["ua_pattern_usage"], inplace=True)
success_user_agents_df = success_user_agents_df.merge(
    success_user_agents_df[["ua_pattern", "usage"]]
    .groupby("ua_pattern")
    .sum()
    .reset_index()
    .rename(columns={"usage": "ua_pattern_usage"}),
    on="ua_pattern",
)

failed_signin_list_df["ua_pattern"] = failed_signin_list_df.UserAgent.str.replace(
    "\d", "x"
)

failed_ua_pattern_usage = (
    failed_signin_list_df[["ua_pattern"]]
    .merge(
        success_user_agents_df[["ua_pattern", "ua_pattern_usage"]].drop_duplicates(),
        on="ua_pattern",
        how="left",
    )
    .fillna(0)
)
failed_signin_list_df["ua_pattern_usage"] = failed_ua_pattern_usage["ua_pattern_usage"]
del failed_ua_pattern_usage

np.histogram(failed_signin_list_df["ua_pattern_usage"], bins=100)
display(
    failed_signin_list_df[["ua_pattern", "ua_pattern_usage"]]
    .drop_duplicates()["ua_pattern_usage"]
    .hvplot.hist(
        bins=100,
        title="Probability distribution for failed login User Agents - likelihood they use a known UA",
        color="orange",
    )
)

In [ ]:
md(
    "UA strings in investigation data that <b>do not</b> appear in the success logon data (top 50)",
    "large",
)
cols = [
    "ResourceDisplayName",
    "AppDisplayName",
    "ResultType",
    "ResultDescription",
    "ua_pattern",
]
obs_data = (
    failed_signin_list_df[
        ~failed_signin_list_df["ua_pattern"].isin(success_user_agents_df.ua_pattern)
    ][[*cols, "TimeGenerated"]]
    .groupby(cols)
    .count()
    .rename(columns={"TimeGenerated": "Login_attempts"})
    .sort_values("Login_attempts", ascending=False)
    .head(50)
    .style.background_gradient(subset=["Login_attempts"], cmap="Reds")
)
display(obs_data)
obs_log.add_observation(
    caption="UA strings in investigation data that **do not** appear in the success logon data (top 50)",
    data=obs_data,
    link=fmt_link("2.5 UserAgent strings"),
    score=2,
)

In [ ]:
md(
    "UA strings in investigation data that <b>do</b> appear in the success logon data (top 50)",
    "large",
)
obs_data = (
    failed_signin_list_df[
        failed_signin_list_df["ua_pattern"].isin(success_user_agents_df["ua_pattern"])
    ][[*cols, "TimeGenerated"]]
    .groupby(cols)
    .count()
    .rename(columns={"TimeGenerated": "Login_attempts"})
    .sort_values("Login_attempts", ascending=False)
    .head(50)
)
display(obs_data)
obs_log.add_observation(
    caption="UA strings in investigation data that **do** appear in the success logon data (top 50)",
    data=obs_data,
    link=fmt_link("2.5 UserAgent strings"),
    score=5,
)

### Observations
<hr>
<p style='background-color: gold; font-size: medium'>
    We should investigate any successful logins from accounts below with these user agents</p>
Note: only data from failed logons data is shown here.
<hr>


In [ ]:
obs_data = (
    failed_signin_list_df[
        failed_signin_list_df["UserAgent"].isin(success_user_agents_df["UserAgent"])
    ][
        [
            "UserAgent",
            "ua_pattern",
            "ResultType",
            "IPAddress",
            "UserPrincipalName",
            "TimeGenerated",
        ]
    ]
    .assign(UserPrincipalName=lambda x: mask_names(x.UserPrincipalName))
    .groupby(["ua_pattern", "ResultType", "UserPrincipalName"])
    .count()
    .rename(columns={"TimeGenerated": "Login_attempts"})
    .sort_values("Login_attempts", ascending=False)
    .head(40)
    .style.background_gradient(subset=["Login_attempts"], cmap="Reds")
)
display(obs_data)
obs_log.add_observation(
    caption="*User login failures with a UA that appears in the success logon data (top 40)",
    data=obs_data,
    link=fmt_link("2.5 UserAgent strings"),
    score=2,
)

### Record observations in dataframe

In [ ]:
failed_signin_list_df["UA_known"] = failed_signin_list_df["ua_pattern"].isin(
    success_user_agents_df["ua_pattern"]
)

failed_ua_pattern_usage = (
    failed_signin_list_df[["ua_pattern"]]
    .merge(
        success_user_agents_df[["ua_pattern", "ua_pattern_usage"]].drop_duplicates(),
        on="ua_pattern",
        how="left",
    )
    .fillna(0)
)
failed_signin_list_df["ua_pattern_usage"] = failed_ua_pattern_usage["ua_pattern_usage"]
del failed_ua_pattern_usage

## 2.6 Distinct UA-AppID pairing

It is likely that different apps accessing email, documents, etc. will have different User Agent strings.
E.g. a mail app may use a specific AppId and have a User Agent string specific to it.

Here we are calculating the frequency at which User Agent strings are used with 
specific AppIds in successful logons.<br>
We then compare our failed logons to see if their UserAgent/AppId pairings match,
and if so, what the relative frequency a given pairing is found in success logons.

In [ ]:
qry = f"""
SigninLogs
| where TimeGenerated > datetime({wednesday}) and TimeGenerated < datetime({thursday})
| where ResultType == 0
| summarize ua_appid_count=count() by UserAgent, AppId
"""

print("Querying UA-AppId statistics...")
file_name = archive_name("success_ua_appid_df", wednesday, thursday)
if not use_caching() or not archive_exists(file_name):
    print("querying sample of current data...")
    success_ua_appid_df = qry_prov.exec_query(qry)
    save_archive(file_name, success_ua_appid_df)
else:
    success_ua_appid_df = get_archive(file_name)
print("done.")

success_ua_appid_df["ua_pattern"] = success_ua_appid_df.UserAgent.str.replace("\d", "x")

success_ua_appid_df = success_ua_appid_df.merge(
    success_ua_appid_df[["ua_pattern", "ua_appid_count"]]
    .groupby("ua_pattern")
    .sum()
    .reset_index()
    .rename(columns={"ua_appid_count": "ua_ptn_appid_count"}),
    on="ua_pattern",
)
success_ua_appid_df["ua_app_id_usage"] = (
    success_ua_appid_df["ua_appid_count"] / success_ua_appid_df.ua_appid_count.sum()
)
success_ua_appid_df["ua_ptn_appid_usage"] = (
    success_ua_appid_df["ua_ptn_appid_count"] / success_ua_appid_df.ua_appid_count.sum()
)


print("Merging into failed logins data")
# Merge usage stats into failed logons
if "ua_ptn_appid_usage" in failed_signin_list_df.columns:
    failed_signin_list_df.drop(columns=["ua_ptn_appid_usage"], inplace=True)
failed_ua_ptn_appid_df = failed_signin_list_df.merge(
    success_ua_appid_df[
        ["ua_pattern", "AppId", "ua_ptn_appid_usage"]
    ].drop_duplicates(),
    on=["ua_pattern", "AppId"],
    how="left",
).fillna(0)
failed_signin_list_df["ua_ptn_appid_usage"] = failed_ua_ptn_appid_df[
    "ua_ptn_appid_usage"
]
display(
    failed_signin_list_df["ua_ptn_appid_usage"].hvplot.hist(
        bins=100,
        title="Failed logins - distribution using UA-AppId pairings found in success logins",
        color="orange",
        height=500,
    )
    + success_ua_appid_df.ua_ptn_appid_usage.hvplot.hist(
        bins=100,
        title="Success logins - distribution of UA-AppId pairings frequency",
        color="blue",
        height=500,
    )
)
del failed_ua_ptn_appid_df

### Showing top 30 failed logons and the frequency that the UA/AppID pairing was found in success logons.

In [ ]:
obs_data = (
    failed_signin_list_df[
        [
            "TimeGenerated",
            "ua_pattern",
            "AppId",
            "UserPrincipalName",
            "ua_ptn_appid_usage",
        ]
    ]
    .assign(UserPrincipalName=lambda x: mask_names(x.UserPrincipalName))
    .groupby(["ua_pattern", "AppId", "UserPrincipalName", "ua_ptn_appid_usage"])
    .count()
    .rename(columns={"TimeGenerated": "count"})
    .sort_values("count", ascending=False)
    .head(30)
    .style.format({"ua_ptn_appid_usage": "{:.5f}"})
    .background_gradient(subset=["count"], cmap="Reds")
)
display(obs_data)
obs_log.add_observation(
    caption="Top login failures by UA pattern and UA/AppId usage (top 30)",
    data=obs_data,
    link=fmt_link("Showing top 30 failed logons and the frequency that the UA/AppID pairing was found in success logons."),
    score=2,
)

## 2.7 Use of legacy authentication protocols

Modern apps and devices use either the 'Browser' or 'Mobile Apps and Desktop clients' protocols.

AAD supports numerous legacy protocols, and are considered less secure.<br>
They are also the protocols typically targeted by attackers since many do not support modern
options such as multi-factor authentication.

In [ ]:
client_app_usage = (
    success_signin_list_df[["TimeGenerated", "ClientAppUsed"]]
    .groupby("ClientAppUsed")
    .count()
    .reset_index()
    .rename(columns={"TimeGenerated": "cli_app_count"})
    .assign(client_app_usage=lambda x: x.cli_app_count / len(success_signin_list_df))
)

legacy_apps = legacy_clients()
legacy_crit = failed_signin_list_df.ClientAppUsed.isin(legacy_apps)
failed_signin_list_df.loc[legacy_crit, "legacy_client"] = True
failed_signin_list_df.loc[~legacy_crit, "legacy_client"] = False


if "client_app_usage" in failed_signin_list_df.columns:
    failed_signin_list_df.drop(columns=["client_app_usage"], inplace=True)
failed_signin_list_df["client_app_usage"] = (
    failed_signin_list_df.merge(
        client_app_usage[["ClientAppUsed", "client_app_usage"]],
        on="ClientAppUsed",
        how="left",
    )
    .fillna(0)
    .client_app_usage
)

group_cols = [
    "ResultType",
    "ResultDescription",
    "UserAgent",
    "AppDisplayName",
    "ClientAppUsed",
    "legacy_client",
    "client_app_usage",
]
obs_data = (
    failed_signin_list_df[legacy_crit][[*group_cols, "TimeGenerated"]]
    .groupby(group_cols)
    .count()
    .rename(columns={"TimeGenerated": "count"})
    .sort_values("count", ascending=False)
)

display(obs_data)
obs_log.add_observation(
    caption="Top login failures by client app for legacy protocols",
    data=obs_data,
    link=fmt_link("2.7 Use of legacy authentication protocols"),
    score=2,
)
ip_log.extend(
    create_ip_entries(
        ip_addresses=(
            failed_signin_list_df[legacy_crit].query("client_app_usage == 0.0").IPAddress.unique()
        ),
        reason="Failed logons using legacy protocols not used in organization",
        section="2.7 Use of legacy authentication protocols",
    )
)

## 2.8 UA Pattern analysis

TODO - we might be able to detect 'artificial' User Agent strings - i.e. UA strings being spoofed by malicious applications.

- Take the UA pattern (i.e. the UA string with version digits removed)
- Take a hash of the pattern - giving a unique value for each pattern
- Take a combination of the following to enable us to match similar but not necessarily identical strings:
  - the sum of ordinal character values of the UA string and rounded to nearest 10
  - a count of the non-alphanumeric characters in the string (giving us a measure of the structure) (rounded to nearest 2)
- Find all UAs that have an equal similarity but dissimilar hashes.
- Do an edit-distance calculation to highlight the changes between the two UA versions.

---
## 2.9 IP Addresses with multiple user names

It should be unusual (VPNs/proxies aside) for many users to share an IP address.

An IPAddress can easily have multiple UserAgents (think different apps on the same device) - so a high number is not necessarily a concern.
In any case, this seems to be somewhat correlated with multiple users, per IP.

We can group by IPAddress and get a count of unique users, useragents and result types

In [ ]:
ip_users_grp = (
    failed_signin_list_df[["IPAddress", "UserPrincipalName", "ResultType", "UserAgent"]]
    .assign(UserPrincipalName=lambda x: mask_names(x.UserPrincipalName))
    .groupby("IPAddress")
    .agg(
        ip_user_count=pd.NamedAgg("UserPrincipalName", "nunique"),
        user_sample=pd.NamedAgg("UserPrincipalName", lambda x: x.unique().tolist()[:5]),
        user_agent_sample=pd.NamedAgg("UserAgent", lambda x: x.unique().tolist()[:5]),
        user_agents=pd.NamedAgg("UserAgent", "nunique"),
        result_types=pd.NamedAgg("ResultType", "nunique"),
    )
)
multi_user_ips = (
    ip_users_grp.query("ip_user_count > 1").reset_index().IPAddress.unique()
)
md(f"{len(multi_user_ips)} IPs with multiple users", "bold")
ips = sorted(list(mask_ips(pd.Series(multi_user_ips)).values))
for idx in range(0, len(ips), 5):
    for ip in ips[idx:idx + 5]:
        print(ip, end=", ")
    print()

md("<br>Top 20", "bold")
obs_data = (
    ip_users_grp.query("ip_user_count > 1")
    .sort_values("ip_user_count", ascending=False)
    .head(20)
    .reset_index()
    .assign(IPAddress=lambda x: mask_ips(x.IPAddress))
)
display(obs_data)
obs_log.add_observation(
    caption="*IP Addresses with multiple user names (top 20)",
    data=obs_data,
    link=fmt_link("2.9 IP Addresses with multiple user names"),
    score=2,
)
ip_log.extend(
    create_ip_entries(
        ip_addresses=multi_user_ips,
        reason="IP Addresses with multiple user names",
        section="2.9 IP Addresses with multiple user names",
    )
)


# Add this data to our Failed user DataFrame
if "ip_user_count" in failed_signin_list_df.columns:
    failed_signin_list_df.drop(columns=["ip_user_count"], inplace=True)

failed_signin_list_df = failed_signin_list_df.merge(
    ip_users_grp.reset_index()[["IPAddress", "ip_user_count"]], on="IPAddress"
)
del ip_users_grp

---
## 2.10 Users with multiple IPs, UserAgents, Locations

This is potentially interesting but it might be common to see users with multiple IP addresses, user agents and often locations.

Unless you see something unusual in the top scorers
for unique distinct IP Addresses, Locations, UserAgents or ResultTypes, we won't use this data further.

In [ ]:
user_ips_grp_clr = (
    failed_signin_list_df[
        [
            "IPAddress",
            "IPBlock_str",
            "UserPrincipalName",
            "ResultType",
            "UserAgent",
            "Location",
        ]
    ]
    .groupby("UserPrincipalName")
    .agg(
        ip_count=pd.NamedAgg("IPAddress", "nunique"),
        ip_sample=pd.NamedAgg("IPAddress", lambda x: x.unique().tolist()[:5]),
        country_count=pd.NamedAgg("Location", "nunique"),
        country_sample=pd.NamedAgg("Location", lambda x: x.unique().tolist()[:5]),
        user_agents=pd.NamedAgg("UserAgent", "nunique"),
        user_agent_sample=pd.NamedAgg("UserAgent", lambda x: x.unique().tolist()[:5]),
        result_types=pd.NamedAgg("ResultType", "nunique"),
        result_type_sample=pd.NamedAgg("ResultType", lambda x: x.unique().tolist()[:5]),
    )
)

def mask_ip_list(series):
    return series.apply(lambda x: [hash_ip(ip) for ip in x])


user_ips_grp = user_ips_grp_clr.reset_index().assign(
    UserPrincipalName=lambda x: mask_names(x.UserPrincipalName),
    ip_sample=lambda x: mask_ip_list(x.ip_sample),
)
md("By ip_count", "bold")
display(user_ips_grp.sort_values("ip_count", ascending=False).head(5))
obs_log.add_observation(
    caption="Failed logins by ip_count",
    data=user_ips_grp.sort_values("ip_count", ascending=False).head(5),
    link=fmt_link("2.10 Users with multiple IPs, UserAgents, Locations"),
    score=1,
)
md("By country_count", "bold")
display(user_ips_grp.sort_values("country_count", ascending=False).head(5))
obs_log.add_observation(
    caption="Failed logins by country_count",
    data=user_ips_grp.sort_values("country_count", ascending=False).head(5),
    link=fmt_link("2.10 Users with multiple IPs, UserAgents, Locations"),
    score=1,
)
del user_ips_grp_clr, user_ips_grp

## 2.11 IP Addresses Identified as Malicious by Azure Active Directory

In [ ]:
mal_ip_desc = (
    "Sign-in was blocked because it came from an IP address with malicious activity."
)
mal_blocked_ips = failed_signin_list_df[
    [
        "IPAddress",
        "UserPrincipalName",
        "ResultType",
        "ResultDescription",
        "UserAgent",
        "Location",
    ]
].query("ResultDescription == @mal_ip_desc")

obs_data = (
    mal_blocked_ips.assign(
        UserPrincipalName=lambda x: mask_names(x.UserPrincipalName),
        IPAddress=lambda x: mask_ips(x.IPAddress),
    )
    .groupby("IPAddress")
    .agg(
        ip_user_count=pd.NamedAgg("UserPrincipalName", "nunique"),
        user_sample=pd.NamedAgg("UserPrincipalName", lambda x: x.unique().tolist()[:5]),
        user_agent_sample=pd.NamedAgg("UserAgent", lambda x: x.unique().tolist()[:5]),
        user_agents=pd.NamedAgg("UserAgent", "nunique"),
        result_types=pd.NamedAgg("ResultType", "nunique"),
    )
)
display(obs_data)
obs_log.add_observation(
    caption="IP Addresses blocked by AAD because malicious",
    data=obs_data,
    link=fmt_link("2.11 IP Addresses Identified as Malicious by Azure Active Directory"),
    score=2,
)
ip_log.extend(
    create_ip_entries(
        ip_addresses=mal_blocked_ips.IPAddress.unique(),
        reason="IP Addresses blocked by AAD because malicious",
        section="2.11 IP Addresses Identified as Malicious by Azure Active Directory",
    )
)


# Add this data to our Failed user DataFrame
if "known_malicious" in failed_signin_list_df.columns:
    failed_signin_list_df.drop(columns=["known_malicious"], inplace=True)

failed_signin_list_df = failed_signin_list_df.merge(
    mal_blocked_ips[["IPAddress"]].assign(known_malicious=True),
    on="IPAddress",
    how="left",
).fillna(False)

del mal_blocked_ips

---

# Part 3 - Data Enrichment

---

## 3.1 Threat Intel Lookup

In [ ]:
from msticpy.sectools import TILookup

# Create a TILookup instance - this caches lookups, so don't recreate on each run.
ti_lookup = TILookup()

In [ ]:
from time import sleep

# Change these two variables if you have data limits on your TI providers
# e.g. 4 lookups per minute - SLICE_SIZE = 4, SLEEP_SECS = 55
SLICE_SIZE = 10 
SLEEP_SECS = 1

# Get all IPs with > 1 reason for inclusion in observations
suspicious_ips = (
    pd.DataFrame(data=set(ip_log))
    .groupby("address")
    .agg({"reason": "nunique"})
    .sort_values("reason", ascending=False)
    .query("reason > 1")
    .reset_index()
)

print("Looking up TI Results...")
ip_slices = [
    suspicious_ips.iloc[idx : idx + SLICE_SIZE]
    for idx in range(0, len(suspicious_ips), SLICE_SIZE)
]
result_slices = []
for ip_slice in tqdm(ip_slices, desc="IoC slices"):
    result_slices.append(ti_lookup.lookup_iocs(data=ip_slice, obs_col="address"))
    print(f"\nWaiting {SLEEP_SECS} sec for next batch", end="")
    for sec in range(SLEEP_SECS):
        print(".", end="")
        sleep(1)

ti_results = pd.concat(result_slices)
print("done")

In [ ]:
ti_results["ti_severity"] = ti_results.Severity.map(
    {"high": 5, "warning": 3, "information": 1}
)
ti_results_grouped = ti_results.groupby("Ioc").agg(
    providers=pd.NamedAgg("Provider", lambda x: x.unique().tolist()),
    details=pd.NamedAgg("Details", lambda x: x.tolist()),
    references=pd.NamedAgg("Reference", lambda x: x.unique().tolist()),
    max_severity=pd.NamedAgg("ti_severity", "max"),
).reset_index()
failed_signins_ti = (
    failed_signin_list_df.merge(
        ti_results_grouped[["Ioc", "providers", "details", "references", "max_severity"]], 
        left_on="IPAddress",
        right_on="Ioc",
        how="left"
    )
    .fillna({"max_severity": 0})
)

failed_signin_list_df["max_ti_severity"] = failed_signins_ti.max_severity

obs_data = (
    failed_signin_list_df
    [[
        "IPAddress",
        "UserPrincipalName",
        "ResultType",
        "ResultDescription",
        "UserAgent",
        "Location",
        "max_ti_severity"
    ]]
    .merge(
        ti_results_grouped[["Ioc", "providers", "details", "references"]], 
        left_on="IPAddress",
        right_on="Ioc",
        how="inner"
    )
    .assign(
        UserPrincipalName=lambda x: mask_names(x.UserPrincipalName),
        IPAddress=lambda x: mask_ips(x.IPAddress),
    )
    .groupby(["IPAddress", "UserAgent"])
    .agg(
        user_count=pd.NamedAgg("UserPrincipalName", "nunique"),
        logins_count=pd.NamedAgg("UserPrincipalName", "count"),
        location_count=pd.NamedAgg("Location", "nunique"),
        max_ti_severity=pd.NamedAgg("max_ti_severity", "max"),
        result_types_count=pd.NamedAgg("ResultType", "nunique"),
        providers=pd.NamedAgg("providers", "first"),
        references=pd.NamedAgg("references", lambda x: x.tolist()),
    )
)
display(obs_data)
obs_log.add_observation(
    caption="*IP Addresses with postive Threat Intel results",
    data=obs_data,
    link=fmt_link("3.1 Threat Intel Lookup"),
    score=5,
)

---

# Part 4 - Bringing the data together

---

## 4.1 Clustering

### Create data features

In [ ]:
from msticpy.sectools.eventcluster import (
    char_ord_score_df,
    crc32_hash_df,
    delim_count_df,
)

# Create features
source_cols = [
    "ResultType",
    "UserAgent",
    "AppId",
    "scheduled",
    "known_ip_block",
    "UA_known",
    "login_loc_prob",
    "device_str",
    "ip_user_count",
    "ua_pattern_usage",
    "ua_pattern",
    "ua_ptn_appid_usage",
    "legacy_client",
    "client_app_usage",
    "max_ti_severity",
    "known_malicious",
]
failed_logins_features = failed_signin_list_df[source_cols].copy()
# failed_logins_features["result_id"]
failed_logins_features["result_type"] = failed_logins_features["ResultType"].astype(
    "int32"
)
failed_logins_features["user_agent"] = failed_logins_features["UserAgent"].astype(
    "category"
)
failed_logins_features["user_agent_scr"] = char_ord_score_df(
    failed_logins_features, "UserAgent"
)
failed_logins_features["user_agent_struc"] = delim_count_df(
    failed_logins_features, "UserAgent"
)
failed_logins_features["user_agent_ptn_scr"] = char_ord_score_df(
    failed_logins_features, "ua_pattern"
)
failed_logins_features["app_id_scr"] = char_ord_score_df(
    failed_logins_features, "AppId"
)
failed_logins_features["device_str"] = char_ord_score_df(
    failed_logins_features, "device_str"
)
# failed_logins_features["device_str_struc"] = delim_count_df(failed_logins_features, "device_str")

input_cols = [
    "scheduled",
    "known_ip_block",
    "UA_known",
    "login_loc_prob",
    "user_agent_scr",
    "user_agent_struc",
    "app_id_scr",
    "device_str",
    "ip_user_count",
    "user_agent_ptn_scr",
    "ua_pattern_usage",
    "ua_ptn_appid_usage",
    "legacy_client",
    "client_app_usage",
    "max_ti_severity",
    "known_malicious",
]
default_cols = [
    "known_ip_block",
    "login_loc_prob",
    "user_agent_struc",
    "ua_pattern_usage",
    "ua_ptn_appid_usage",
    "legacy_client",
    "client_app_usage",
    "max_ti_severity",
]

### Select cluster parameters

In [ ]:
w_layout = widgets.Layout(width="50%")
w_style = {"description_width": "200px"}
cluster_strength = widgets.FloatSlider(
    description="Cluster strength",
    min=0.1,
    max=5.0,
    value=1.5,
    step=0.1,
    layout=w_layout,
    style=w_style,
)

min_cluster_size = widgets.IntSlider(
    description="Min cluster size",
    min=1,
    max=30,
    step=1,
    value=10,
    layout=w_layout,
    style=w_style,
)

cluster_props = widgets.SelectMultiple(
    description="Cluster Properties",
    options=input_cols,
    value=default_cols,
    layout=widgets.Layout(width="50%", height="200px"),
    style=w_style,
)
cluster_datasets = widgets.SelectMultiple(
    description="Datasets",
    options=failed_logins_features.ResultType.unique(),
    value=list(failed_logins_features.ResultType.unique()),
    layout=widgets.Layout(width="50%", height="200px"),
    style=w_style,
)
display(cluster_strength)
display(min_cluster_size)
display(cluster_props)
display(cluster_datasets)

### Run clustering algorithm (DBSCAN)

In [ ]:
import numpy as np
from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

failed_logins_x = failed_logins_features[
    failed_logins_features.ResultType.isin(cluster_datasets.value)
]
failed_logins_data_x = failed_signin_list_df[
    failed_signin_list_df.ResultType.isin(cluster_datasets.value)
]
X = failed_logins_x[list(cluster_props.value)]
X = StandardScaler().fit_transform(X)

# #############################################################################
# Compute DBSCAN
print("Fitting cluster...")
db = DBSCAN(eps=cluster_strength.value, min_samples=min_cluster_size.value).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

# #############################################################################
# Plot result
import matplotlib.pyplot as plt

# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = labels == k

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(
        xy[:, 0],
        xy[:, 1],
        "o",
        markerfacecolor=tuple(col),
        markeredgecolor="k",
        markersize=14,
    )

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(
        xy[:, 0],
        xy[:, 1],
        "o",
        markerfacecolor=tuple(col),
        markeredgecolor="k",
        markersize=6,
    )

plt.title("Estimated number of clusters: %d" % n_clusters_)
plt.show()

### Create a DataFrame with results

In [ ]:
from IPython.display import HTML

# Used to normalize these two scores
max_known_ip_block = failed_logins_x["known_ip_block"].max()
max_known_ua_usage = failed_logins_x["ua_pattern_usage"].max()
max_ip_user_count = failed_logins_x["ip_user_count"].max()

cluster_groups = []
for label in unique_labels:
    cluster_group = pd.Series()
    cluster_group.name = label

    failed_df_x = failed_logins_x[labels == label]
    cluster_group["app_ids"] = "\n".join(list(failed_df_x["AppId"].unique()))
    cluster_group["user_agents"] = "\n".join(list(failed_df_x["ua_pattern"].unique()))
    cluster_group["num_users"] = failed_logins_data_x[labels == label][
        "UserPrincipalName"
    ].nunique()
    cluster_group["num_user_agents"] = failed_df_x["ua_pattern"].nunique()
    cluster_group["num_ap_ids"] = failed_df_x["AppId"].nunique()
    cluster_group["num_ips"] = failed_logins_data_x[labels == label].IPAddress.nunique()
    cluster_group["cluster_size"] = len(failed_df_x)

    cluster_group["med_known_ip_blk"] = (
        failed_df_x["known_ip_block"].median() / max_known_ip_block
    )
    cluster_group["med_loc_prob"] = failed_df_x["login_loc_prob"].median()
    cluster_group["med_scheduled"] = failed_df_x["scheduled"].median()
    cluster_group["med_ip_user_count"] = failed_df_x["ip_user_count"].median()
    cluster_group["med_ua_known"] = failed_df_x["UA_known"].median()
    cluster_group["ua_pattern_usage"] = failed_df_x["ua_pattern_usage"].median()
    cluster_group["ua_ptn_appid_usage"] = failed_df_x["ua_ptn_appid_usage"].median()
    cluster_group["client_app_usage"] = failed_df_x["client_app_usage"].median()
    cluster_group["legacy_client"] = failed_df_x["legacy_client"].median()
    cluster_group["max_ti_severity"] = failed_df_x["max_ti_severity"].median()

    add_scores = (
        cluster_group["med_known_ip_blk"]  # smaller is more suspicious
        + cluster_group["med_loc_prob"]  # smaller is more suspicious
        + (1 - cluster_group["med_scheduled"])  # 1 is more suspicious
        # + (1 - cluster_group["med_ip_user_count"])  # larger is more suspicious
        # + cluster_group["med_ua_known"]  # 0 is more suspicious
        + cluster_group["ua_ptn_appid_usage"]
        + cluster_group["client_app_usage"]
        + cluster_group["ua_pattern_usage"]  # smaller is more suspicious
        + cluster_group["max_ti_severity"]
        + 1 - cluster_group["legacy_client"]
    )
    #     print(
    #         cluster_group["med_known_ip_blk"],  # smaller is more suspicious
    #         cluster_group["med_loc_prob"],  # smaller is more suspicious
    #         #         (1 - cluster_group["med_scheduled"]),  # 1 is more suspicious
    #         #         (1 - cluster_group["med_ip_user_count"]),  # larger is more suspicious
    #         cluster_group["med_ua_known"],  # 0 is more suspicious
    #         cluster_group["ua_ptn_appid_usage"],
    #         cluster_group["client_app_usage"],
    #         cluster_group["ua_pattern_usage"]
    #     )
    add_scores = add_scores if add_scores else 1
    cluster_group["combined_score"] = 1 / add_scores
    #     print(min(cluster_group["combined_score"], 10))

    cluster_groups.append(cluster_group)

cluster_df = pd.DataFrame(cluster_groups)

### Display overall stats for the clusters

> **Note** the `combined_score` is a rough approximation.<br>
> It is mostly based on the known locations, known ip blocks and known user agent data.<br>
> In most cases this combination will indicate the most suspicious clusters but 
> there may be other overriding factors (e.g. an attack from a known location).<br>
> You should examine the data carefully to determine the possible threat posed.

In [ ]:
display(cluster_df.sort_values("combined_score", ascending=False).head(20))

## Plot of clusters - clusters nearer zero are more suspicious

- Vertical axis is the median probability of the logon IP coming from a known location
- Horizontal axis is the median probability of the logon IP belonging to a know IP block
- Circle size represents the cluster size
- Color is combined score - deeper color == higher "suspiciousness" score.

> Note: multiple clusters may occupy the same place on the chart

In [ ]:
cluster_df.hvplot.scatter(
    x="med_known_ip_blk",
    y="med_loc_prob",
    size="cluster_size",
    color="combined_score",
    cmap="reds",
    line_color="black",
    height=500,
    width=800,
)

## 4.2 Browse clustered events

The browser below groups related events together making it easier to analyze the data.

In [ ]:
clus_select_options = [
    (
        label,
        cluster_df.loc[label].cluster_size,
        cluster_df.loc[label].combined_score,
        cluster_df.loc[label].num_users,
    )
    for label in unique_labels
]
clus_select_options = sorted(clus_select_options, key=lambda x: x[2], reverse=True)
option_dict = {
    (
        f"Cluster {item[0]:2}, "
        + f" size: {item[1]:4}, "
        + f" score: {item[2]:6.2f}, "
        + f" users: {item[3]:3}"
    ): item[0]
    for item in clus_select_options
}

select_cluster = widgets.Select(
    options=option_dict,
    description="Select cluster to view",
    layout=widgets.Layout(width="50%", height="200px"),
    style={"description_width": "150px"},
)

group_cols = [
    "IPAddress",
    "Location",
    "UserPrincipalName",
    "ResultDescription",
    "AppDisplayName",
    "ua_pattern",
    "UA_known",
    "known_ip_block",
    "scheduled",
    "login_loc_prob",
    "ua_pattern_usage",
    "max_ti_severity",
]


def get_grouped_data(label):
    return (
        failed_logins_data_x[labels == label][["TimeGenerated", *group_cols]]
        .groupby(group_cols)
        .agg(
            start=pd.NamedAgg("TimeGenerated", "min"),
            end=pd.NamedAgg("TimeGenerated", "max"),
            events=pd.NamedAgg("TimeGenerated", "count"),
        )
        .reset_index()
        .sort_values("events", ascending=False)[["start", "end", "events", *group_cols]]
    )


def disp_header(label):
    if label == -1:
        disp_head_id.update(Markdown("**Unclustered events/noise points**"))
    else:
        disp_head_id.update(Markdown(
            f"**Cluster {label}   Score {cluster_df.loc[label].combined_score:0.3f}**"
        ))


def show_logins(change):
    label = change.get("new")
    disp_header(label)
    if disp_df_id:
        disp_stat_id.update(pd.DataFrame(cluster_df.loc[label]).T)
        disp_df_id.update(get_grouped_data(label))


md("Note - cluster with label '-1' is the group of remaining unclustered events.")
select_cluster.observe(show_logins, names="value")
display(select_cluster)

first_label = next(iter(option_dict.values()))
disp_head_id = display("", display_id=True)
disp_header(first_label)
disp_stat_id = display(pd.DataFrame(cluster_df.loc[first_label]), display_id=True)
disp_df_id = display(get_grouped_data(first_label), display_id=True)

## 4.3 Browse investigation observations

In [ ]:
opts = list(obs_log.observation_list.keys())
select_obs = widgets.Select(options=opts, style=style, layout=layout)


def display_observation(obs, disp_ids=None):
    """Display the observation."""
    desc = obs.description or ""
    link = f"[Go to notebook section: *{obs.link}*](#{obs.link})" if obs.link else ""
    if disp_ids is None:
        disp_ids = [
            display(Markdown(f"### {obs.caption}"), display_id=True),
            display(Markdown(desc), display_id=True),
            display(Markdown(f"Score: {obs.score}"), display_id=True),
            display(Markdown(link), display_id=True),
            display(obs.data, display_id=True),
            display(Markdown("---"), display_id=True),
        ]
    else:
        disp_ids[0].update(Markdown(f"### {obs.caption}"))
        disp_ids[1].update(Markdown(desc))
        disp_ids[2].update(Markdown(f"Score: {obs.score}"))
        disp_ids[3].update(Markdown(link))
        disp_ids[4].update(obs.data)
        disp_ids[5].update(Markdown("---"))
    return disp_ids


def disp_obs(change):
    obs_key = change.get("new")
    display_observation(obs_log.observation_list.get(obs_key), disp_ids)


select_obs.observe(disp_obs, names="value")
display(select_obs)
disp_ids = display_observation(next(iter(obs_log.observation_list.values())))
select_obs.value = next(iter(obs_log.observation_list.keys()))

## 4.4 Results query browser

This section lets you display the full results of the signin failures. You can query on 
any of the DataFrame properties, remove duplicates and select subsets of columns to view.

Type in a query expression into the `Data query` box and hit the `Apply query` button.
- strings must be quoted
- you can use and and or to group sub-expressions
- for string operations (e.g. "contains") use the pandas `str` accessor. E.g.
  - UserPrincipalName.str.contains("ian")
  
The `Avail columns` control (from the left) is just a list of columns - 
making it easier to copy/paste column names into a query.<br>
The `Show columns` control lets you select which columns to display.<br>
`Apply` applies the current query (if any), column selection, duplicate removal and Top N settings.<br>
`Show top` restricts the output to the top N rows. 0 means all rows.<br>
`Drop duplicates` tries to remove duplicate columns. In cases where the column value 
(e.g. LocationDetails) is a dictionary or other unhashable type this will not work.<br>
`Results to clipboard` copies a tab-separated list of the current results to the clipboard


In [ ]:
w_style = {"description_width": "100px"}
right_align = widgets.Layout(align_self="center")
query_text = widgets.Textarea(
    description="Data query",
    layout=widgets.Layout(width="70%", height="100px"),
    style=w_style,
)
col_names = widgets.Textarea(
    description="Avail columns",
    value="\n".join(sorted(failed_signin_list_df.columns)),
    placeholder="max_ti_severity == 5",
    layout=widgets.Layout(width="30%", height="150px"),
    style=w_style,
    disabled=True,
)
col_select = widgets.SelectMultiple(
    description="Show columns",
    options=list(sorted(failed_signin_list_df.columns)),
    value=list(sorted(failed_signin_list_df.columns)),
    layout=widgets.Layout(width="40%", height="150px"),
    style=w_style,
)
head_len = widgets.BoundedIntText(
    value=0,
    min=0,
    max=len(failed_signin_list_df),
    description="Show top",
    layout=widgets.Layout(align_self="center", width="200px"),
    style=w_style,
)
apply_btn = widgets.Button(description="Apply query", layout=right_align)
copy_btn = widgets.Button(description="Results to clipboard", layout=right_align)
no_dups = widgets.Checkbox(description="Drop duplicates", value=True, layout=right_align)


def get_filtered_events():
    df_display = failed_signin_list_df
    if query_text.value:
        qry = query_text.value.replace("\n", " ")
        df_display = df_display.query(qry)
    if head_len.value > 0:
        df_display = df_display.head(head_len.value)
    df_display = df_display[list(col_select.value)]
    if no_dups.value:
        try:
            df_display = df_display.drop_duplicates()
        except TypeError:
            pass
    return df_display


def filter_events(change):
    del change
    df_display = get_filtered_events()
    disp_filt_df_id.update(df_display)


def to_clipboard(change):
    del change
    get_filtered_events().to_clipboard()


apply_btn.on_click(filter_events)
copy_btn.on_click(to_clipboard)
actions = widgets.HBox(children=[apply_btn, no_dups, head_len, copy_btn])
display(
    widgets.VBox(
        [
            widgets.HBox([query_text, col_names]),
            col_select,
            actions,
        ]
    )
)
md("<hr>")
disp_filt_df_id = display(failed_signin_list_df.head(10), display_id=True)

---
<br><br><br><br><br><br>
# ----- End of Part 4 ------
<br><br><br><br><br><br>

---

# Appendices

## Check memory usage

In [ ]:
import sys


def show_mem_usage(namespace, df_only=True, thresh_kb=100):
    vars = list(namespace.keys())
    df_memory = []
    for name in vars:
        val = namespace.get(name)
        if df_only and isinstance(val, pd.DataFrame):
            df_memory.append((name, type(val).__name__, val.memory_usage().sum()))
        else:
            df_memory.append((name, type(val).__name__, sys.getsizeof(val, 666)))

    for name, v_type, mem_usage in sorted(df_memory, key=lambda x: x[2], reverse=True):
        if mem_usage > thresh_kb * 1024:
            print(f"{name:40} {v_type:40} {mem_usage /  1024:>20,.1f} kb")


show_mem_usage(globals(), df_only=False)

## Save DataFrames

In [ ]:
# Save
from pathlib import Path
from datetime import datetime

data_path = f"pwd_spray_data{datetime.now().strftime('%Y-%m-%dT%H-%M')}"

if not Path(data_path).exists():
    Path(data_path).mkdir()
print("Saving dataFrames in Notebook")
print("-" * 50)
current_vars = list(locals().keys())
for var_name in current_vars:
    if isinstance(locals()[var_name], pd.DataFrame) and not var_name.startswith("_"):
        print("saving ", data_path + "/" + var_name + ".pkl")
        try:
            locals()[var_name].to_pickle(data_path + "/" + var_name + ".pkl")
        except MemoryError:
            print("Failed to save", var_name)